In [1]:
import pandas as pd

# --- 1. TENTUKAN NAMA FILE DAN PARAMETER ---

# Ganti ini dengan nama file CSV kamu yang sebenarnya
file_wdi = 'WDICSV-rev.csv'
file_hdi = 'human-development-index-(hdi)-by-country-2025.csv'
file_suicide = 'suicide-rate-by-country-2025.csv'



In [2]:
# Indikator WDI yang ingin kamu ambil (berdasarkan chat sebelumnya)
wdi_indicators_to_keep = [
    'SP.POP.0014.TO.ZS', # Populasi 0-14
    'SP.POP.1564.TO.ZS', # Populasi 15-64
    'SP.POP.65UP.TO.ZS'  # Populasi 65+
    'SL.UEM.TOTL.ZS'     # <-- INDIKATOR BARU: Unemployment Total
]

# Rentang tahun utama untuk analisis
# (Meskipun data suicide hanya 2019-2021, kita ambil rentang lebih luas dulu)
years_to_keep = [2019, 2020, 2021, 2022, 2023]

# --- 2. MEMBACA SEMUA FILE ---
try:
    df_wdi_raw = pd.read_csv(file_wdi)
    df_hdi_raw = pd.read_csv(file_hdi)
    df_suicide_raw = pd.read_csv(file_suicide)
except FileNotFoundError as e:
    print(f"Error: File tidak ditemukan. Pastikan nama file sudah benar.")
    print(e)
    # Hentikan eksekusi jika file tidak ada
    exit()

print("Semua file berhasil dibaca.")

# --- 3. PROSES DATA WDICSV ---
print("Memproses data WDI...")

# 3.1. Filter hanya indikator yang kita butuhkan
df_wdi_filtered = df_wdi_raw[df_wdi_raw['Indicator Code'].isin(wdi_indicators_to_keep)].copy()

# 3.2. Melt data WDI
# Kolom identitas (ID)
id_vars_wdi = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']
# Kolom nilai (tahun) - ambil semua kolom selain ID
value_vars_wdi = [col for col in df_wdi_filtered.columns if col not in id_vars_wdi]

df_wdi_melted = df_wdi_filtered.melt(id_vars=id_vars_wdi,
                                   value_vars=value_vars_wdi,
                                   var_name='Year',
                                   value_name='Value')

# 3.3. Bersihkan dan filter tahun
df_wdi_melted['Year'] = pd.to_numeric(df_wdi_melted['Year'], errors='coerce')
df_wdi_melted = df_wdi_melted.dropna(subset=['Year']) # Hapus jika 'Year' bukan angka
df_wdi_melted['Year'] = df_wdi_melted['Year'].astype(int)
df_wdi_melted = df_wdi_melted[df_wdi_melted['Year'].isin(years_to_keep)]

# 3.4. Pivot data WDI agar indikator menjadi kolom
# Ini mengubah 'Indicator Code' dari baris menjadi kolom
df_wdi_processed = df_wdi_melted.pivot_table(index=['Country Name', 'Year'],
                                           columns='Indicator Code',
                                           values='Value').reset_index()
df_wdi_processed.columns.name = None # Menghapus nama indeks kolom

print("Data WDI selesai diproses.")

# --- 4. PROSES DATA HDI ---
print("Memproses data HDI...")

# 4.1. Tentukan kolom HDI yang akan di-melt
hdi_cols_to_melt = [
    'HumanDevelopmentIndex_2019',
    'HumanDevelopmentIndex_2020',
    'HumanDevelopmentIndex_2021',
    'HumanDevelopmentIndex_2022',
    'HumanDevelopmentIndex_2023'
]
# Filter hanya kolom yang ada di file
hdi_cols_exist = [col for col in hdi_cols_to_melt if col in df_hdi_raw.columns]

# 4.2. Melt data HDI
df_hdi_melted = df_hdi_raw.melt(id_vars=['country'],
                                value_vars=hdi_cols_exist,
                                var_name='Indicator',
                                value_name='HDI')

# 4.3. Ekstrak tahun dari 'Indicator'
df_hdi_melted['Year'] = df_hdi_melted['Indicator'].str.extract(r'(\d{4})')
df_hdi_melted['Year'] = pd.to_numeric(df_hdi_melted['Year']).astype(int)

# 4.4. Siapkan untuk merge (ganti nama kolom & pilih kolom)
df_hdi_processed = df_hdi_melted.rename(columns={'country': 'Country Name'})
df_hdi_processed = df_hdi_processed[['Country Name', 'Year', 'HDI']]

print("Data HDI selesai diproses.")

# --- 5. PROSES DATA SUICIDE RATE ---
print("Memproses data Suicide Rate...")

# 5.1. Tentukan kolom Suicide Rate yang akan di-melt (HANYA YANG UMUM)
suicide_cols_to_melt = [
    'SuicideRateCountries_2019',
    'SuicideRateCountries_2020',
    'SuicideRateCountries_2021'
    # Tambahkan 'SuicideRateCountries_2022' jika ada di file-mu
]
# Filter hanya kolom yang ada di file
suicide_cols_exist = [col for col in suicide_cols_to_melt if col in df_suicide_raw.columns]

# 5.2. Melt data Suicide Rate
df_suicide_melted = df_suicide_raw.melt(id_vars=['country'],
                                        value_vars=suicide_cols_exist,
                                        var_name='Indicator',
                                        value_name='Suicide Rate')

# 5.3. Ekstrak tahun
df_suicide_melted['Year'] = df_suicide_melted['Indicator'].str.extract(r'(\d{4})')
df_suicide_melted['Year'] = pd.to_numeric(df_suicide_melted['Year']).astype(int)

# 5.4. Siapkan untuk merge
df_suicide_processed = df_suicide_melted.rename(columns={'country': 'Country Name'})
df_suicide_processed = df_suicide_processed[['Country Name', 'Year', 'Suicide Rate']]

print("Data Suicide Rate selesai diproses.")



Semua file berhasil dibaca.
Memproses data WDI...
Data WDI selesai diproses.
Memproses data HDI...
Data HDI selesai diproses.
Memproses data Suicide Rate...
Data Suicide Rate selesai diproses.


In [3]:
# --- 6. GABUNGKAN (MERGE) SEMUA DATA ---
print("Menggabungkan semua data...")

# 6.1. Mulai dengan data WDI
df_merged = df_wdi_processed.copy()

# 6.2. Merge dengan HDI
# 'how=outer' berarti kita menyimpan semua data, meskipun ada tahun/negara yang
# tidak cocok di kedua dataset.
df_merged = pd.merge(df_merged, df_hdi_processed, on=['Country Name', 'Year'], how='outer')

# 6.3. Merge dengan Suicide Rate
df_merged = pd.merge(df_merged, df_suicide_processed, on=['Country Name', 'Year'], how='outer')

print("Data berhasil digabungkan.")

# --- 7. FINALISASI DATA ---

# 7.1. Filter rentang tahun akhir
# Analisis gabungan (HDI vs Suicide) hanya bisa dilakukan untuk tahun 2019-2021
# karena data suicide rate-mu (berdasarkan list kolom) hanya sampai 2021.
# Jika kamu punya data suicide 2022, ubah ini.
analysis_years = [2019, 2020, 2021]
df_final = df_merged[df_merged['Year'].isin(analysis_years)].copy()

# 7.2. Hapus baris yang semua datanya kosong (kecuali Country & Year)
data_cols = [col for col in df_final.columns if col not in ['Country Name', 'Year']]
df_final = df_final.dropna(subset=data_cols, how='all')

# 7.3. Simpan ke file CSV baru
output_file = 'dataset_gabungan_final.csv'
df_final.to_csv(output_file, index=False)

print("\n--- SELESAI ---")
print(f"Data gabungan berhasil disimpan ke: {output_file}")
print("\n5 baris pertama data gabungan:")
print(df_final.head())
print("\nInformasi data:")
df_final.info()

Menggabungkan semua data...
Data berhasil digabungkan.

--- SELESAI ---
Data gabungan berhasil disimpan ke: dataset_gabungan_final.csv

5 baris pertama data gabungan:
                  Country Name  Year  SP.POP.0014.TO.ZS  SP.POP.1564.TO.ZS  \
0                  Afghanistan  2019          44.576832          53.047015   
1                  Afghanistan  2020          44.224104          53.405162   
2                  Afghanistan  2021          43.908125          53.739325   
5  Africa Eastern and Southern  2019          41.666633          55.176225   
6  Africa Eastern and Southern  2020          41.381733          55.426098   

     HDI  Suicide Rate  
0  0.492          4.10  
1  0.488          3.63  
2  0.473          3.60  
5    NaN           NaN  
6    NaN           NaN  

Informasi data:
<class 'pandas.core.frame.DataFrame'>
Index: 885 entries, 0 to 1468
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 

In [7]:
!pip install altair


  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached narwhals-2.9.0-py3-none-any.whl.metadata (11 kB)
Using cached altair-5.5.0-py3-none-any.whl (731 kB)
Using cached narwhals-2.9.0-py3-none-any.whl (422 kB)

   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   -------------------- ------------------- 1/2 [altair]
   -------------------- ------------------- 1/2 [altair]
   -------------------- ------------------- 1/2 [altair]
   ---------------------------------------- 2/2 [altair]



In [2]:
import pandas as pd
import altair as alt

# --- 1. Define File Names ---
file_hdi = 'human-development-index-(hdi)-by-country-2025.csv'
file_suicide = 'suicide-rate-by-country-2025.csv'
file_metadata = 'WDICountry.csv'

print("Starting visualization script for Slide 1 (v2)...")

# --- 2. Load Data ---
try:
    df_hdi_raw = pd.read_csv(file_hdi)
    df_suicide_raw = pd.read_csv(file_suicide)
    df_meta = pd.read_csv(file_metadata)
    print("All 3 data files loaded successfully.")
except Exception as e:
    print(f"An error occurred while reading files: {e}")
    exit()

# --- 3. Process & Merge Data (Same as before) ---
# Process HDI
hdi_cols_to_melt = ['HumanDevelopmentIndex_2019', 'HumanDevelopmentIndex_2020', 'HumanDevelopmentIndex_2021']
hdi_cols_exist = [col for col in hdi_cols_to_melt if col in df_hdi_raw.columns]
df_hdi_melted = df_hdi_raw.melt(id_vars=['country'], value_vars=hdi_cols_exist, var_name='HDI_Indicator', value_name='HDI')
df_hdi_melted['Year'] = df_hdi_melted['HDI_Indicator'].str.extract(r'(\d{4})').astype(int)
df_hdi_processed = df_hdi_melted[['country', 'Year', 'HDI']]

# Process Suicide
suicide_cols_to_melt = ['SuicideRateCountries_2019', 'SuicideRateCountries_2020', 'SuicideRateCountries_2021']
suicide_cols_exist = [col for col in suicide_cols_to_melt if col in df_suicide_raw.columns]
df_suicide_melted = df_suicide_raw.melt(id_vars=['country'], value_vars=suicide_cols_exist, var_name='Suicide_Indicator', value_name='Suicide Rate')
df_suicide_melted['Year'] = df_suicide_melted['Suicide_Indicator'].str.extract(r'(\d{4})').astype(int)
df_suicide_processed = df_suicide_melted[['country', 'Year', 'Suicide Rate']]

# Process Metadata
df_meta_processed = df_meta[['Table Name', 'Region', 'Income Group']].copy()
df_meta_processed = df_meta_processed.dropna(subset=['Region', 'Income Group'])
df_meta_processed = df_meta_processed.rename(columns={'Table Name': 'country'})

# Merge
df_merged = pd.merge(df_hdi_processed, df_suicide_processed, on=['country', 'Year'], how='inner')
df_final = pd.merge(df_merged, df_meta_processed, on='country', how='inner')

# Clean and filter for 2021
df_final['HDI'] = pd.to_numeric(df_final['HDI'], errors='coerce')
df_final['Suicide Rate'] = pd.to_numeric(df_final['Suicide Rate'], errors='coerce')
df_final = df_final.dropna(subset=['HDI', 'Suicide Rate', 'Region', 'Income Group'])
df_2021 = df_final[df_final['Year'] == 2021].copy()

if df_2021.empty:
    print("Error: No data for 2021.")
    exit()
else:
    print(f"Data merge complete. Using {len(df_2021)} data points from 2021.")

# --- 4. Create Plot 1: Boxplot (Income Group vs Suicide) + Jitter Points (Unchanged) ---
try:
    income_order = ['Low income', 'Lower middle income', 'Upper middle income', 'High income']
    
    base_plot1 = alt.Chart(df_2021).encode(
        x=alt.X('Income Group', sort=income_order, title='Income Group'),
        y=alt.Y('Suicide Rate', title='Suicide Rate (per 100k)')
    )
    boxplot_layer = base_plot1.mark_boxplot(size=50, opacity=0.4, outliers=True)
    points_layer = base_plot1.mark_point(filled=True, size=60, opacity=0.7).encode(
        color=alt.Color('Region', title='Continent'),
        tooltip=['country', 'Region', 'Income Group', 'Suicide Rate']
    )
    chart1 = (boxplot_layer + points_layer).properties(
        title='Suicide Rate by Income Group and Continent (2021)',
        width=600,
        height=400
    ).interactive()

    chart1_file = 'slide1_boxplot_income_vs_suicide_v2.json'
    chart1.save(chart1_file)
    print(f"Success: Chart 1 (Boxplot) saved to {chart1_file}")

except Exception as e:
    print(f"Error creating Chart 1 (Boxplot): {e}")

# --- 5. Create Plot 2: Scatter (HDI vs Suicide) + Threshold Line + Regression Line (MODIFIED) ---
try:
    average_suicide_rate = df_2021['Suicide Rate'].mean()
    
    # Base chart for scatter
    base_plot2 = alt.Chart(df_2021).encode(
        x=alt.X('HDI', title='Human Development Index (HDI)', scale=alt.Scale(zero=False)),
        y=alt.Y('Suicide Rate', title='Suicide Rate (per 100k)')
    )
    
    # Layer 1: Scatter plot points
    scatter_layer = base_plot2.mark_circle(size=80, opacity=0.8).encode(
        color=alt.Color('Region', title='Continent'),
        tooltip=['country', 'Region', 'HDI', 'Suicide Rate']
    )
    
    # Layer 2: Horizontal Threshold Line (Red Dashed)
    threshold_line = alt.Chart(pd.DataFrame({'y': [average_suicide_rate]})).mark_rule(
        color='red',
        strokeDash=[5,5], # Dashed line
        size=2
    ).encode(y='y')
    
    # Text label for the threshold line
    threshold_text = alt.Chart(pd.DataFrame({
        'y': [average_suicide_rate], 
        'label': [f'Global Average: {average_suicide_rate:.2f}']
    })
    ).mark_text(
        align='left',
        baseline='bottom',
        color='red',
        dx=5  # Horizontal offset
    ).encode(
        y='y',
        text='label'
    )

    # Layer 3: Linear Regression Line (Blue Dashed) - INI YANG BARU
    regression_line = base_plot2.transform_regression(
        'HDI', 'Suicide Rate', method='linear'
    ).mark_line(
        color='blue',
        strokeDash=[5,5], # Dashed line
        size=2
    )

    # Combine all layers
    chart2 = (scatter_layer + threshold_line + threshold_text + regression_line).properties(
        title='HDI vs. Suicide Rate by Continent (2021)',
        width=700,
        height=450
    ).interactive()

    chart2_file = 'slide1_scatter_hdi_vs_suicide_v2.json'
    chart2.save(chart2_file)
    print(f"Success: Chart 2 (Scatter with Regression) saved to {chart2_file}")

except Exception as e:
    print(f"Error creating Chart 2 (Scatter): {e}")

Starting visualization script for Slide 1 (v2)...
All 3 data files loaded successfully.
Data merge complete. Using 163 data points from 2021.
Success: Chart 1 (Boxplot) saved to slide1_boxplot_income_vs_suicide_v2.json
Success: Chart 2 (Scatter with Regression) saved to slide1_scatter_hdi_vs_suicide_v2.json


In [4]:
pip install altair_saver vega_datasets selenium


   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ------- -------------------------------- 1.8/9.7 MB 8.4 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.7 MB 10.1 MB/s eta 0:00:01
   ------------------------- -------------- 6.3/9.7 MB 10.2 MB/s eta 0:00:01
   ---------------------------------- ----- 8.4/9.7 MB 10.4 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 9.9 MB/s  0:00:00
   ---------------------------------------- 0.0/844.5 kB ? eta -:--:--
   ---------------------------------------- 844.5/844.5 kB 9.4 MB/s  0:00:00

   ------------- --------------------------  4/12 [outcome]
   ---------------- -----------------------  5/12 [trio]
   ---------------- -----------------------  5/12 [trio]
   ---------------- -----------------------  5/12 [trio]
   ---------------- -----------------------  5/12 [trio]
   ---------------- -----------------------  5/12 [trio]
   ----------------------- ----------------  7/12 [trio-w

In [6]:
pip install altair==4.2.2 altair_saver==0.5.0


  Using cached entrypoints-0.4-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/813.6 kB ? eta -:--:--
   ------------------------------------- - 786.4/813.6 kB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 813.6/813.6 kB 2.7 MB/s  0:00:00
Using cached entrypoints-0.4-py3-none-any.whl (5.3 kB)

   ---------------------------------------- 0/3 [toolz]
  Attempting uninstall: altair
   ---------------------------------------- 0/3 [toolz]
    Found existing installation: altair 5.5.0
   ---------------------------------------- 0/3 [toolz]
    Uninstalling altair-5.5.0:
   ---------------------------------------- 0/3 [toolz]
      Successfully uninstalled altair-5.5.0
   ---------------------------------------- 0/3 [toolz]
   -------------------------- ------------- 2/3 [altair]
   -------------------------- ------------- 2/3 [altair]
   -------------------------- ------------- 2/3 [altair]
   -------------------------- ------------- 2/3

In [9]:
!pip install -U altair_viewer
!pip install -U altair_viewer


In [10]:
import pandas as pd
import altair as alt
import altair_saver


# --- 1. Define File Names (FIXED) ---
# Nama file diperbaiki, disesuaikan dengan yang kamu upload
file_hdi = 'human-development-index-(hdi)-by-country-2025.csv'
file_suicide = 'suicide-rate-by-country-2025.csv'
file_metadata = 'WDICountry.csv'

print("Starting visualization script for Slide 1 (v4 - PNG attempt)...")

# --- 2. Load Data ---
try:
    df_hdi_raw = pd.read_csv(file_hdi)
    df_suicide_raw = pd.read_csv(file_suicide)
    df_meta = pd.read_csv(file_metadata)
    print("All 3 data files loaded successfully.")
except Exception as e:
    print(f"An error occurred while reading files: {e}")
    exit()

# --- 3. Process & Merge Data ---
print("Processing and merging data...")
# (Kode proses dan merge sama persis seperti sebelumnya)
hdi_cols_to_melt = ['HumanDevelopmentIndex_2019', 'HumanDevelopmentIndex_2020', 'HumanDevelopmentIndex_2021']
hdi_cols_exist = [col for col in hdi_cols_to_melt if col in df_hdi_raw.columns]
df_hdi_melted = df_hdi_raw.melt(id_vars=['country'], value_vars=hdi_cols_exist, var_name='HDI_Indicator', value_name='HDI')
df_hdi_melted['Year'] = df_hdi_melted['HDI_Indicator'].str.extract(r'(\d{4})').astype(int)
df_hdi_processed = df_hdi_melted[['country', 'Year', 'HDI']]

suicide_cols_to_melt = ['SuicideRateCountries_2019', 'SuicideRateCountries_2020', 'SuicideRateCountries_2021']
suicide_cols_exist = [col for col in suicide_cols_to_melt if col in df_suicide_raw.columns]
df_suicide_melted = df_suicide_raw.melt(id_vars=['country'], value_vars=suicide_cols_exist, var_name='Suicide_Indicator', value_name='Suicide Rate')
df_suicide_melted['Year'] = df_suicide_melted['Suicide_Indicator'].str.extract(r'(\d{4})').astype(int)
df_suicide_processed = df_suicide_melted[['country', 'Year', 'Suicide Rate']]

df_meta_processed = df_meta[['Table Name', 'Region', 'Income Group']].copy()
df_meta_processed = df_meta_processed.dropna(subset=['Region', 'Income Group'])
df_meta_processed = df_meta_processed.rename(columns={'Table Name': 'country'})

df_merged = pd.merge(df_hdi_processed, df_suicide_processed, on=['country', 'Year'], how='inner')
df_final = pd.merge(df_merged, df_meta_processed, on='country', how='inner')

df_final['HDI'] = pd.to_numeric(df_final['HDI'], errors='coerce')
df_final['Suicide Rate'] = pd.to_numeric(df_final['Suicide Rate'], errors='coerce')
df_final = df_final.dropna(subset=['HDI', 'Suicide Rate', 'Region', 'Income Group'])
df_2021 = df_final[df_final['Year'] == 2021].copy()

if df_2021.empty:
    print("Error: No data for 2021.")
    exit()
else:
    print(f"Data merge complete. Using {len(df_2021)} data points from 2021.")

# --- 4. Create Plot 1: Boxplot (Income Group vs Suicide) ---
try:
    income_order = ['Low income', 'Lower middle income', 'Upper middle income', 'High income']
    
    base_plot1 = alt.Chart(df_2021).encode(
        x=alt.X('Income Group', sort=income_order, title='Income Group'),
        y=alt.Y('Suicide Rate', title='Suicide Rate (per 100k)')
    )
    boxplot_layer = base_plot1.mark_boxplot(size=50, opacity=0.4, outliers=True)
    points_layer = base_plot1.mark_point(filled=True, size=60, opacity=0.7).encode(
        color=alt.Color('Region', title='Continent'),
        tooltip=['country', 'Region', 'Income Group', 'Suicide Rate']
    )
    chart1 = (boxplot_layer + points_layer).properties(
        title='Suicide Rate by Income Group and Continent (2021)',
        width=600,
        height=400
    ).interactive()

    # Save as JSON (this is what gets displayed)
    chart1_file_json = 'slide1_boxplot_income_vs_suicide.json'
    chart1.save(chart1_file_json)
    print(f"Success: Chart 1 (Boxplot) saved to {chart1_file_json} for display.")
    
    # Attempt to save as PNG
    try:
        from altair_saver import save
        chart1_file_png = 'slide1_boxplot_income_vs_suicide.png'
        save(chart1, chart1_file_png)
        print(f"Success: Chart 1 (Boxplot) also saved to {chart1_file_png}.")
    except ImportError:
        print("Info: 'altair_saver' library not found. Cannot save as PNG.")
    except Exception as e:
        print(f"Error saving Chart 1 as PNG: {e}")

except Exception as e:
    print(f"Error creating Chart 1 (Boxplot): {e}")

# --- 5. Create Plot 2: Scatter (HDI vs Suicide) + Threshold Line + Regression Line ---
try:
    average_suicide_rate = df_2021['Suicide Rate'].mean()
    
    base_plot2 = alt.Chart(df_2021).encode(
        x=alt.X('HDI', title='Human Development Index (HDI)', scale=alt.Scale(zero=False)),
        y=alt.Y('Suicide Rate', title='Suicide Rate (per 100k)')
    )
    
    scatter_layer = base_plot2.mark_circle(size=80, opacity=0.8).encode(
        color=alt.Color('Region', title='Continent'),
        tooltip=['country', 'Region', 'HDI', 'Suicide Rate']
    )
    
    threshold_line = alt.Chart(pd.DataFrame({'y': [average_suicide_rate]})).mark_rule(
        color='red', strokeDash=[5,5], size=2
    ).encode(y='y')
    
    threshold_text = alt.Chart(pd.DataFrame({
        'y': [average_suicide_rate], 
        'label': [f'Global Average: {average_suicide_rate:.2f}']
    })).mark_text(
        align='left', baseline='bottom', color='red', dx=5
    ).encode(y='y', text='label')

    regression_line = base_plot2.transform_regression(
        'HDI', 'Suicide Rate', method='linear'
    ).mark_line(
        color='blue', strokeDash=[5,5], size=2
    )

    chart2 = (scatter_layer + threshold_line + threshold_text + regression_line).properties(
        title='HDI vs. Suicide Rate by Continent (2021)',
        width=700,
        height=450
    ).interactive()

    # Save as JSON (this is what gets displayed)
    chart2_file_json = 'slide1_scatter_hdi_vs_suicide.json'
    chart2.save(chart2_file_json)
    print(f"Success: Chart 2 (Scatter) saved to {chart2_file_json} for display.")

    # Attempt to save as PNG
    try:
        from altair_saver import save
        chart2_file_png = 'slide1_scatter_hdi_vs_suicide.png'
        save(chart2, chart2_file_png)
        print(f"Success: Chart 2 (Scatter) also saved to {chart2_file_png}.")
    except ImportError:
        print("Info: 'altair_saver' library not found. Cannot save as PNG.")
    except Exception as e:
        print(f"Error saving Chart 2 as PNG: {e}")

except Exception as e:
    print(f"Error creating Chart 2 (Scatter): {e}")

Starting visualization script for Slide 1 (v4 - PNG attempt)...
All 3 data files loaded successfully.
Processing and merging data...
Data merge complete. Using 163 data points from 2021.
Success: Chart 1 (Boxplot) saved to slide1_boxplot_income_vs_suicide.json for display.
Error saving Chart 1 as PNG: No matches for version='5.20.1' among ['4.0.2', '4.8.1', '4.17.0'].
Often this can be fixed by updating altair_viewer:
    pip install -U altair_viewer
Success: Chart 2 (Scatter) saved to slide1_scatter_hdi_vs_suicide.json for display.
Error saving Chart 2 as PNG: No matches for version='5.20.1' among ['4.0.2', '4.8.1', '4.17.0'].
Often this can be fixed by updating altair_viewer:
    pip install -U altair_viewer


In [11]:
pip install altair_saver

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install -U altair_viewer

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install vl-convert-python

   ---------------------------------------- 0.0/31.3 MB ? eta -:--:--
   - -------------------------------------- 1.3/31.3 MB 7.5 MB/s eta 0:00:05
   ---- ----------------------------------- 3.4/31.3 MB 9.2 MB/s eta 0:00:04
   ------- -------------------------------- 5.5/31.3 MB 9.3 MB/s eta 0:00:03
   --------- ------------------------------ 7.3/31.3 MB 9.3 MB/s eta 0:00:03
   ------------ --------------------------- 9.4/31.3 MB 9.2 MB/s eta 0:00:03
   -------------- ------------------------- 11.3/31.3 MB 9.3 MB/s eta 0:00:03
   ----------------- ---------------------- 13.4/31.3 MB 9.2 MB/s eta 0:00:02
   ------------------- -------------------- 15.2/31.3 MB 9.2 MB/s eta 0:00:02
   --------------------- ------------------ 17.0/31.3 MB 9.2 MB/s eta 0:00:02
   ------------------------ --------------- 18.9/31.3 MB 9.2 MB/s eta 0:00:02
   ------------------------- -------------- 20.2/31.3 MB 9.2 MB/s eta 0:00:02
   ----------------------------- ---------- 22.8/31.3 MB 9.2 MB/s eta 0:00:01

In [16]:
import vl_convert as vlc
import json

print("Mencoba menyimpan PNG menggunakan 'vl-convert'...")

try:
    # --- 1. Muat file JSON scatter plot ---
    with open('slide1_scatter_hdi_vs_suicide.json', 'r') as f:
        chart_scatter_json = json.load(f)
    
    # Convert ke PNG
    png_data_scatter = vlc.vegalite_to_png(vl_spec=chart_scatter_json)
    
    # Simpan file PNG
    with open("hdi_vs_suicide_plot.png", "wb") as f:
        f.write(png_data_scatter)
    print("Berhasil menyimpan: hdi_vs_suicide_plot.png")


    # --- 2. Muat file JSON boxplot ---
    with open('slide1_boxplot_income_vs_suicide.json', 'r') as f:
        chart_boxplot_json = json.load(f)

    # Convert ke PNG
    png_data_boxplot = vlc.vegalite_to_png(vl_spec=chart_boxplot_json)

    # Simpan file PNG
    with open("income_vs_suicide_plot.png", "wb") as f:
        f.write(png_data_boxplot)
    print("Berhasil menyimpan: income_vs_suicide_plot.png")

except Exception as e:
    print(f"\nTerjadi error: {e}")
    print("Pastikan kamu sudah menjalankan 'pip install vl-convert-python'")

Mencoba menyimpan PNG menggunakan 'vl-convert'...
Berhasil menyimpan: hdi_vs_suicide_plot.png

Terjadi error: Vega-Lite to PNG conversion failed:
Error: Duplicate signal name: "param_9_Income_Group"
    at l (https://cdn.jsdelivr.net/npm/vega-util@2.0.0/+esm:7:324)
    at hi.addSignal (https://cdn.jsdelivr.net/npm/vega-parser@7.0.0/+esm:7:45557)
    at te (https://cdn.jsdelivr.net/npm/vega-parser@7.0.0/+esm:7:4228)
    at https://cdn.jsdelivr.net/npm/vega-parser@7.0.0/+esm:7:38925
    at Array.forEach (<anonymous>)
    at gi (https://cdn.jsdelivr.net/npm/vega-parser@7.0.0/+esm:7:38913)
    at Module.$i (https://cdn.jsdelivr.net/npm/vega-parser@7.0.0/+esm:7:43022)
    at vegaToView (ext:<anon>:3:24)
    at vegaToSvg (ext:<anon>:33:16)
    at vegaLiteToSvg_v6_1 (ext:<anon>:23:12)
Pastikan kamu sudah menjalankan 'pip install vl-convert-python'


In [17]:
import vl_convert as vlc
import json

# Nama file JSON input (yang sudah kita buat)
json_file_name = 'slide1_boxplot_income_vs_suicide.json'

# Nama file PNG output (yang kamu inginkan)
png_file_name = 'income_vs_suicide_plot.png'

print(f"Mencoba memuat '{json_file_name}'...")

try:
    # --- 1. Muat file JSON boxplot ---
    with open(json_file_name, 'r') as f:
        chart_boxplot_json = json.load(f)
    print("File JSON berhasil dimuat.")

    # --- 2. Convert ke PNG ---
    print("Mengonversi ke PNG...")
    png_data_boxplot = vlc.vegalite_to_png(vl_spec=chart_boxplot_json)

    # --- 3. Simpan file PNG ---
    with open(png_file_name, "wb") as f:
        f.write(png_data_boxplot)
    
    print(f"\n--- BERHASIL ---")
    print(f"Plot berhasil disimpan sebagai: {png_file_name}")

except FileNotFoundError:
    print(f"\nERROR: File tidak ditemukan: '{json_file_name}'")
    print("Pastikan file JSON itu ada di folder yang sama dengan script ini.")
except Exception as e:
    print(f"\nTerjadi error: {e}")
    print("Pastikan kamu sudah menjalankan 'pip install vl-convert-python'")

Mencoba memuat 'slide1_boxplot_income_vs_suicide.json'...
File JSON berhasil dimuat.
Mengonversi ke PNG...

Terjadi error: Vega-Lite to PNG conversion failed:
Error: Duplicate signal name: "param_9_Income_Group"
    at l (https://cdn.jsdelivr.net/npm/vega-util@2.0.0/+esm:7:324)
    at hi.addSignal (https://cdn.jsdelivr.net/npm/vega-parser@7.0.0/+esm:7:45557)
    at te (https://cdn.jsdelivr.net/npm/vega-parser@7.0.0/+esm:7:4228)
    at https://cdn.jsdelivr.net/npm/vega-parser@7.0.0/+esm:7:38925
    at Array.forEach (<anonymous>)
    at gi (https://cdn.jsdelivr.net/npm/vega-parser@7.0.0/+esm:7:38913)
    at Module.$i (https://cdn.jsdelivr.net/npm/vega-parser@7.0.0/+esm:7:43022)
    at vegaToView (ext:<anon>:3:24)
    at vegaToSvg (ext:<anon>:33:16)
    at vegaLiteToSvg_v6_1 (ext:<anon>:23:12)
Pastikan kamu sudah menjalankan 'pip install vl-convert-python'


In [18]:
import pandas as pd
import altair as alt

# --- 1. Define File Names ---
file_hdi = 'human-development-index-(hdi)-by-country-2025.csv'
file_suicide = 'suicide-rate-by-country-2025.csv'
file_metadata = 'WDICountry.csv'

print("--- Langkah 1: Membuat Ulang JSON (Versi Statis) ---")

# --- 2. Load Data ---
try:
    df_hdi_raw = pd.read_csv(file_hdi)
    df_suicide_raw = pd.read_csv(file_suicide)
    df_meta = pd.read_csv(file_metadata)
    print("Data berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat file: {e}")
    exit()

# --- 3. Process & Merge Data ---
print("Memproses data...")
# (Kode proses data disingkat, sama seperti sebelumnya)
hdi_cols_to_melt = ['HumanDevelopmentIndex_2019', 'HumanDevelopmentIndex_2020', 'HumanDevelopmentIndex_2021']
hdi_cols_exist = [col for col in hdi_cols_to_melt if col in df_hdi_raw.columns]
df_hdi_melted = df_hdi_raw.melt(id_vars=['country'], value_vars=hdi_cols_exist, var_name='HDI_Indicator', value_name='HDI')
df_hdi_melted['Year'] = df_hdi_melted['HDI_Indicator'].str.extract(r'(\d{4})').astype(int)
df_hdi_processed = df_hdi_melted[['country', 'Year', 'HDI']]

suicide_cols_to_melt = ['SuicideRateCountries_2019', 'SuicideRateCountries_2020', 'SuicideRateCountries_2021']
suicide_cols_exist = [col for col in suicide_cols_to_melt if col in df_suicide_raw.columns]
df_suicide_melted = df_suicide_raw.melt(id_vars=['country'], value_vars=suicide_cols_exist, var_name='Suicide_Indicator', value_name='Suicide Rate')
df_suicide_melted['Year'] = df_suicide_melted['Suicide_Indicator'].str.extract(r'(\d{4})').astype(int)
df_suicide_processed = df_suicide_melted[['country', 'Year', 'Suicide Rate']]

df_meta_processed = df_meta[['Table Name', 'Region', 'Income Group']].copy()
df_meta_processed = df_meta_processed.dropna(subset=['Region', 'Income Group'])
df_meta_processed = df_meta_processed.rename(columns={'Table Name': 'country'})

df_merged = pd.merge(df_hdi_processed, df_suicide_processed, on=['country', 'Year'], how='inner')
df_final = pd.merge(df_merged, df_meta_processed, on='country', how='inner')

df_final['HDI'] = pd.to_numeric(df_final['HDI'], errors='coerce')
df_final['Suicide Rate'] = pd.to_numeric(df_final['Suicide Rate'], errors='coerce')
df_final = df_final.dropna(subset=['HDI', 'Suicide Rate', 'Region', 'Income Group'])
df_2021 = df_final[df_final['Year'] == 2021].copy()
print("Data selesai diproses.")

# --- 4. Create Plot 1 (Boxplot) - VERSI STATIS ---
# (Ini bagian yang diubah)
try:
    income_order = ['Low income', 'Lower middle income', 'Upper middle income', 'High income']
    
    base_plot1 = alt.Chart(df_2021).encode(
        x=alt.X('Income Group', sort=income_order, title='Income Group'),
        y=alt.Y('Suicide Rate', title='Suicide Rate (per 100k)')
    )
    boxplot_layer = base_plot1.mark_boxplot(size=50, opacity=0.4, outliers=True)
    
    points_layer = base_plot1.mark_point(filled=True, size=60, opacity=0.7).encode(
        color=alt.Color('Region', title='Continent')
        # Tooltip dihilangkan untuk menghindari bug
    )
    
    chart1 = (boxplot_layer + points_layer).properties(
        title='Suicide Rate by Income Group and Continent (2021)',
        width=600,
        height=400
    ) # .interactive() dihilangkan untuk menghindari bug

    # Simpan file JSON (menimpa file lama)
    chart1_file = 'slide1_boxplot_income_vs_suicide.json'
    chart1.save(chart1_file)
    print(f"\n--- BERHASIL (Langkah 1) ---")
    print(f"File JSON statis baru telah disimpan ke: {chart1_file}")
    print("Sekarang, jalankan script konversi PNG-mu lagi.")

except Exception as e:
    print(f"Error membuat chart statis: {e}")

--- Langkah 1: Membuat Ulang JSON (Versi Statis) ---
Data berhasil dimuat.
Memproses data...
Data selesai diproses.

--- BERHASIL (Langkah 1) ---
File JSON statis baru telah disimpan ke: slide1_boxplot_income_vs_suicide.json
Sekarang, jalankan script konversi PNG-mu lagi.


In [19]:
import vl_convert as vlc
import json

# Nama file JSON input (yang baru saja kamu buat ulang)
json_file_name = 'slide1_boxplot_income_vs_suicide.json'

# Nama file PNG output
png_file_name = 'income_vs_suicide_plot.png'

print(f"\n--- Langkah 2: Mengonversi JSON ke PNG ---")
print(f"Mencoba memuat '{json_file_name}'...")

try:
    with open(json_file_name, 'r') as f:
        chart_boxplot_json = json.load(f)
    print("File JSON berhasil dimuat.")

    print("Mengonversi ke PNG...")
    png_data_boxplot = vlc.vegalite_to_png(vl_spec=chart_boxplot_json)

    with open(png_file_name, "wb") as f:
        f.write(png_data_boxplot)
    
    print(f"\n--- BERHASIL (Langkah 2) ---")
    print(f"Plot berhasil disimpan sebagai: {png_file_name}")

except Exception as e:
    print(f"\nTerjadi error saat konversi: {e}")


--- Langkah 2: Mengonversi JSON ke PNG ---
Mencoba memuat 'slide1_boxplot_income_vs_suicide.json'...
File JSON berhasil dimuat.
Mengonversi ke PNG...

--- BERHASIL (Langkah 2) ---
Plot berhasil disimpan sebagai: income_vs_suicide_plot.png


In [21]:
import pandas as pd
import altair as alt

# --- 1. Define File Names ---
file_hdi = 'human-development-index-(hdi)-by-country-2025.csv'
file_suicide = 'suicide-rate-by-country-2025.csv'
file_metadata = 'WDICountry.csv'

print("--- Memulai script visualisasi (perbaikan xOffset) ---")

# --- 2. Load Data ---
try:
    df_hdi_raw = pd.read_csv(file_hdi)
    df_suicide_raw = pd.read_csv(file_suicide)
    df_meta = pd.read_csv(file_metadata)
    print("Semua 3 file data berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat file: {e}")
    exit()

# --- 3. Process & Merge Data ---
print("Memproses dan menggabungkan data...")
hdi_cols_to_melt = ['HumanDevelopmentIndex_2019', 'HumanDevelopmentIndex_2020', 'HumanDevelopmentIndex_2021']
hdi_cols_exist = [col for col in hdi_cols_to_melt if col in df_hdi_raw.columns]
df_hdi_melted = df_hdi_raw.melt(id_vars=['country'], value_vars=hdi_cols_exist, var_name='HDI_Indicator', value_name='HDI')
df_hdi_melted['Year'] = df_hdi_melted['HDI_Indicator'].str.extract(r'(\d{4})').astype(int)
df_hdi_processed = df_hdi_melted[['country', 'Year', 'HDI']]

suicide_cols_to_melt = ['SuicideRateCountries_2019', 'SuicideRateCountries_2020', 'SuicideRateCountries_2021']
suicide_cols_exist = [col for col in suicide_cols_to_melt if col in df_suicide_raw.columns]
df_suicide_melted = df_suicide_raw.melt(id_vars=['country'], value_vars=suicide_cols_exist, var_name='Suicide_Indicator', value_name='Suicide Rate')
df_suicide_melted['Year'] = df_suicide_melted['Suicide_Indicator'].str.extract(r'(\d{4})').astype(int)
df_suicide_processed = df_suicide_melted[['country', 'Year', 'Suicide Rate']]

df_meta_processed = df_meta[['Table Name', 'Region', 'Income Group']].copy()
df_meta_processed = df_meta_processed.dropna(subset=['Region', 'Income Group'])
df_meta_processed = df_meta_processed.rename(columns={'Table Name': 'country'})

df_merged = pd.merge(df_hdi_processed, df_suicide_processed, on=['country', 'Year'], how='inner')
df_final = pd.merge(df_merged, df_meta_processed, on='country', how='inner')

df_final['HDI'] = pd.to_numeric(df_final['HDI'], errors='coerce')
df_final['Suicide Rate'] = pd.to_numeric(df_final['Suicide Rate'], errors='coerce')
df_final = df_final.dropna(subset=['HDI', 'Suicide Rate', 'Region', 'Income Group'])
df_2021 = df_final[df_final['Year'] == 2021].copy()
print(f"Data 2021 siap (total {len(df_2021)} negara).")


# --- 4. Create Plot 1 (Boxplot) - DIPERBAIKI ---
try:
    income_order = ['Low income', 'Lower middle income', 'Upper middle income', 'High income']
    
    base_plot1 = alt.Chart(df_2021).encode(
        x=alt.X('Income Group', sort=income_order, title='Income Group'),
        y=alt.Y('Suicide Rate', title='Suicide Rate (per 100k)')
    )
    
    # Layer 1: Boxplot (di tengah, lebih tipis)
    boxplot_layer = base_plot1.mark_boxplot(
        size=30,  # Box lebih tipis
        opacity=0.6, 
        outliers=True
        # xOffset DIHAPUS karena menyebabkan error
    )
    
    # Layer 2: Points (diperkecil dan digeser ke kanan)
    points_layer = base_plot1.mark_point(
        filled=True,
        size=20,  # Titik diperkecil
        opacity=0.7,
        xOffset=25 # Geser ke kanan (hanya titiknya)
    ).encode(
        color=alt.Color('Region', title='Continent')
        # Tooltip dan .interactive() sengaja dihilangkan
    )
    
    chart1 = (boxplot_layer + points_layer).properties(
        title='Suicide Rate by Income Group and Continent (2021)',
        width=600,
        height=400
    )

    chart1_file = 'slide1_boxplot_STATIC.json'
    chart1.save(chart1_file)
    print(f"Success: Chart 1 (Boxplot Statis) disimpan ke {chart1_file}")

except Exception as e:
    print(f"Error membuat Chart 1 (Boxplot): {e}")

# --- 5. Create Plot 2 (Scatter) - (Tidak Berubah) ---
try:
    base_plot2 = alt.Chart(df_2021).encode(
        x=alt.X('HDI', title='Human Development Index (HDI)', scale=alt.Scale(zero=False)),
        y=alt.Y('Suicide Rate', title='Suicide Rate (per 100k)')
    )
    
    scatter_layer = base_plot2.mark_circle(size=80, opacity=0.8).encode(
        color=alt.Color('Region', title='Continent')
    )
    
    regression_line = base_plot2.transform_regression(
        'HDI', 'Suicide Rate', method='linear'
    ).mark_line(
        color='black', 
        size=2
    )

    chart2 = (scatter_layer + regression_line).properties(
        title='HDI vs. Suicide Rate by Continent (2021)',
        width=700,
        height=450
    )

    chart2_file = 'slide1_scatter_STATIC.json'
    chart2.save(chart2_file)
    print(f"Success: Chart 2 (Scatter Statis) disimpan ke {chart2_file}")

except Exception as e:
    print(f"Error membuat Chart 2 (Scatter): {e}")

--- Memulai script visualisasi (perbaikan xOffset) ---
Semua 3 file data berhasil dimuat.
Memproses dan menggabungkan data...
Data 2021 siap (total 163 negara).
Success: Chart 1 (Boxplot Statis) disimpan ke slide1_boxplot_STATIC.json
Success: Chart 2 (Scatter Statis) disimpan ke slide1_scatter_STATIC.json


In [22]:
import vl_convert as vlc
import json

print("--- Memulai Konversi JSON Statis ke PNG ---")

# --- Daftar Plot untuk Dikonversi ---
plots_to_convert = [
    {
        "json_in": "slide1_boxplot_STATIC.json",
        "png_out": "income_vs_suicide_plot.png"
    },
    {
        "json_in": "slide1_scatter_STATIC.json",
        "png_out": "hdi_vs_suicide_plot.png"
    }
]

# --- Loop Konversi ---
for plot in plots_to_convert:
    json_file = plot["json_in"]
    png_file = plot["png_out"]
    
    print(f"\nMemproses: '{json_file}'")
    
    try:
        # 1. Muat file JSON statis
        with open(json_file, 'r') as f:
            chart_json = json.load(f)
        print(f"'{json_file}' berhasil dimuat.")

        # 2. Konversi ke PNG
        print(f"Mengonversi ke PNG...")
        png_data = vlc.vegalite_to_png(vl_spec=chart_json)

        # 3. Simpan file PNG
        with open(png_file, "wb") as f:
            f.write(png_data)
        
        print(f"--- BERHASIL disimpan sebagai: {png_file} ---")

    except FileNotFoundError:
        print(f"ERROR: File tidak ditemukan: '{json_file}'")
    except Exception as e:
        print(f"ERROR saat memproses '{json_file}': {e}")
        print("Pastikan kamu sudah menjalankan 'pip install vl-convert-python'")

print("\n--- Konversi Selesai ---")

--- Memulai Konversi JSON Statis ke PNG ---

Memproses: 'slide1_boxplot_STATIC.json'
'slide1_boxplot_STATIC.json' berhasil dimuat.
Mengonversi ke PNG...
--- BERHASIL disimpan sebagai: income_vs_suicide_plot.png ---

Memproses: 'slide1_scatter_STATIC.json'
'slide1_scatter_STATIC.json' berhasil dimuat.
Mengonversi ke PNG...
--- BERHASIL disimpan sebagai: hdi_vs_suicide_plot.png ---

--- Konversi Selesai ---


In [13]:
import pandas as pd
import altair as alt

# --- 1. Define File Names ---
file_hdi = 'human-development-index-(hdi)-by-country-2025.csv'
file_suicide = 'suicide-rate-by-country-2025.csv'
file_metadata = 'WDICountry.csv'

print("--- Memulai script Perbaikan Boxplot (Label Horizontal) ---")

# --- 2. Load Data ---
try:
    df_hdi_raw = pd.read_csv(file_hdi)
    df_suicide_raw = pd.read_csv(file_suicide)
    df_meta = pd.read_csv(file_metadata)
    print("Semua 3 file data berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat file: {e}")
    exit()

# --- 3. Process & Merge Data ---
# (Kode ini sama seperti sebelumnya, untuk menyiapkan data)
print("Memproses dan menggabungkan data...")
hdi_cols_to_melt = ['HumanDevelopmentIndex_2019', 'HumanDevelopmentIndex_2020', 'HumanDevelopmentIndex_2021']
hdi_cols_exist = [col for col in hdi_cols_to_melt if col in df_hdi_raw.columns]
df_hdi_melted = df_hdi_raw.melt(id_vars=['country'], value_vars=hdi_cols_exist, var_name='HDI_Indicator', value_name='HDI')
df_hdi_melted['Year'] = df_hdi_melted['HDI_Indicator'].str.extract(r'(\d{4})').astype(int)
df_hdi_processed = df_hdi_melted[['country', 'Year', 'HDI']]

suicide_cols_to_melt = ['SuicideRateCountries_2019', 'SuicideRateCountries_2020', 'SuicideRateCountries_2021']
suicide_cols_exist = [col for col in suicide_cols_to_melt if col in df_suicide_raw.columns]
df_suicide_melted = df_suicide_raw.melt(id_vars=['country'], value_vars=suicide_cols_exist, var_name='Suicide_Indicator', value_name='Suicide Rate')
df_suicide_melted['Year'] = df_suicide_melted['Suicide_Indicator'].str.extract(r'(\d{4})').astype(int)
df_suicide_processed = df_suicide_melted[['country', 'Year', 'Suicide Rate']]

df_meta_processed = df_meta[['Table Name', 'Region', 'Income Group']].copy()
df_meta_processed = df_meta_processed.dropna(subset=['Region', 'Income Group'])
df_meta_processed = df_meta_processed.rename(columns={'Table Name': 'country'})

df_merged = pd.merge(df_hdi_processed, df_suicide_processed, on=['country', 'Year'], how='inner')
df_final = pd.merge(df_merged, df_meta_processed, on='country', how='inner')

df_final['HDI'] = pd.to_numeric(df_final['HDI'], errors='coerce')
df_final['Suicide Rate'] = pd.to_numeric(df_final['Suicide Rate'], errors='coerce')
df_final = df_final.dropna(subset=['HDI', 'Suicide Rate', 'Region', 'Income Group'])
df_2021 = df_final[df_final['Year'] == 2021].copy()
print(f"Data 2021 siap (total {len(df_2021)} negara).")


# --- 4. Create Plot 1 (Boxplot) - LABEL HORIZONTAL ---
try:
    income_order = ['Low income', 'Lower middle income', 'Upper middle income', 'High income']

    base_plot1 = alt.Chart(df_2021).encode(
        # --- PERBAIKAN DI SINI: labelAngle=0 ---
        x=alt.X('Income Group',
                sort=income_order,
                title='Income Group',
                axis=alt.Axis(labelAngle=0) # Paksa label jadi horizontal
               ),
        y=alt.Y('Suicide Rate', title='Suicide Rate (per 100k)')
    )

    # Layer 1: Boxplot (di tengah, lebih tipis)
    boxplot_layer = base_plot1.mark_boxplot(
        size=30,
        opacity=0.6,
        outliers=True
    )

    # Layer 2: Points (diperkecil dan digeser ke kanan)
    points_layer = base_plot1.mark_point(
        filled=True,
        size=20,
        opacity=0.7,
        xOffset=25 # Geser ke kanan
    ).encode(
        color=alt.Color('Region', title='Continent')
    )

    chart1 = (boxplot_layer + points_layer).properties(
        title='Suicide Rate by Income Group and Continent (2021)',
        width=600,
        height=400 # Mungkin perlu disesuaikan jika labelnya jadi mepet
    )

    chart1_file = 'slide1_boxplot_STATIC_horizontal.json'
    chart1.save(chart1_file)
    print(f"Success: Boxplot (Label Horizontal) disimpan ke {chart1_file}")

except Exception as e:
    print(f"Error membuat Chart 1 (Boxplot): {e}")

--- Memulai script Perbaikan Boxplot (Label Horizontal) ---
Semua 3 file data berhasil dimuat.
Memproses dan menggabungkan data...
Data 2021 siap (total 163 negara).
Success: Boxplot (Label Horizontal) disimpan ke slide1_boxplot_STATIC_horizontal.json


c:\software\environments\deep_learning\environments\deep_learning_cv\Lib\site-packages\altair\utils\core.py:384: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.

c:\software\environments\deep_learning\environments\deep_learning_cv\Lib\site-packages\altair\utils\core.py:384: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.

c:\software\environments\deep_learning\environments\deep_learning_cv\Lib\site-packages\altair\utils\core.py:384: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.



In [14]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Mengatur 'template' default agar background-nya putih
pio.templates.default = "plotly_white"

print("--- Memulai script Plot 1: Boxplot Time Series (Versi PLOTLY) ---")

# --- 1. Tentukan Nama File ---
file_master = 'dataset_master_lengkap.csv'
file_metadata = 'WDICountry.csv' # File metadata untuk data 'Region' & 'Income Group'

# --- 2. Muat Data ---
try:
    df_master = pd.read_csv(file_master)
    df_meta = pd.read_csv(file_metadata)
    print(f"Berhasil memuat {file_master} dan {file_metadata}")
except FileNotFoundError as e:
    print(f"Error: File tidak ditemukan. {e}")
    exit()
except Exception as e:
    print(f"Error saat memuat file: {e}")
    exit()

# --- 3. Gabungkan Data Master dengan Metadata ---
# Siapkan metadata: ambil 'Table Name', 'Region', dan 'Income Group'
df_meta_processed = df_meta[['Table Name', 'Region', 'Income Group']].dropna()

# Gabungkan!
df_with_meta = pd.merge(
    df_master,
    df_meta_processed,
    left_on='Country Name',
    right_on='Table Name',
    how='left' 
)
print("Berhasil menggabungkan data master dengan metadata (Region & Income Group).")

# --- 4. Proses Data ---
col_rate = 'Suicide Rate'
col_year = 'Year'
col_country = 'Country Name'
col_region = 'Region'
col_income = 'Income Group'

# Cek apakah semua kolom ada
required_cols = [col_rate, col_year, col_country, col_region, col_income]
if not all(col in df_with_meta.columns for col in required_cols):
    print(f"Error: Gagal menggabungkan atau kolom tidak ditemukan.")
    print(f"Kolom yang ada: {df_with_meta.columns.tolist()}")
    exit()
else:
    print("Semua kolom (termasuk Region & Income Group) ditemukan.")

# Filter hanya tahun yang kita mau
years_to_plot = [2019, 2020, 2021]
df_filtered = df_with_meta[df_with_meta[col_year].isin(years_to_plot)].copy()

# Bersihkan data
df_filtered = df_filtered.dropna(subset=[col_rate, col_income, col_region])

# Tentukan urutan kategori Income Group
income_order = ['Low income', 'Lower middle income', 'Upper middle income', 'High income']
# Ubah jadi tipe 'category' agar urutannya benar di plot
df_filtered[col_income] = pd.Categorical(df_filtered[col_income], categories=income_order, ordered=True)

if df_filtered.empty:
    print("Error: Tidak ada data valid untuk tahun 2019-2021 setelah dibersihkan.")
    exit()

print(f"Data 2019-2021 (dengan Region & Income) siap untuk di-plot.")


# --- 5. Buat Visualisasi (Plotly Boxplot + Slider + Warna) ---
try:
    print("Membuat plot Plotly...")
    fig = px.box(
        df_filtered,
        x=col_income,
        y=col_rate,
        animation_frame="Year",     # Slider Tahun
        color=col_region,           # <-- Warna berdasarkan Benua
        points='all',               # <-- Tampilkan semua titik (seperti di plot statis)
        hover_name="Country Name",
        title="Suicide Rate by Income Group and Continent (2019-2021)",
        category_orders={col_income: income_order} # Pastikan urutan sumbu X benar
    )
    
    # Atur label sumbu
    fig.update_layout(
        xaxis_title="Income Group",
        yaxis_title="Suicide Rate (per 100k)",
        legend_title="Continent" # Judul legenda
    )
    
    # --- 6. Menyimpan Plot ---
    
    # Simpan sebagai HTML (Interaktif, Pasti Berhasil)
    html_file = 'slide1_boxplot_TIMESERIES.html'
    fig.write_html(html_file)
    print(f"\n--- BERHASIL (HTML) ---")
    print(f"Plot Boxplot INTERAKTIF (berwarna) disimpan ke: {html_file}")

    # Coba Simpan sebagai PNG (Statis)
    try:
        png_file = 'slide1_boxplot_TIMESERIES.png'
        fig.write_image(png_file, width=1000, height=550)
        print(f"\n--- BERHASIL (PNG) ---")
        print(f"Gambar plot (PNG berwarna) berhasil disimpan ke: {png_file}")
    except ValueError as e_png:
        print(f"\n--- INFO (PNG Gagal) ---")
        print(f"Gagal menyimpan PNG: {e_png}")
        print("Pastikan 'kaleido' sudah ter-install: pip install kaleido")
        
except Exception as e_fig:
    print(f"Error membuat Peta Plotly: {e_fig}")

--- Memulai script Plot 1: Boxplot Time Series (Versi PLOTLY) ---
Berhasil memuat dataset_master_lengkap.csv dan WDICountry.csv
Berhasil menggabungkan data master dengan metadata (Region & Income Group).
Semua kolom (termasuk Region & Income Group) ditemukan.
Data 2019-2021 (dengan Region & Income) siap untuk di-plot.
Membuat plot Plotly...

--- BERHASIL (HTML) ---
Plot Boxplot INTERAKTIF (berwarna) disimpan ke: slide1_boxplot_TIMESERIES.html

--- INFO (PNG Gagal) ---
Gagal menyimpan PNG: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Pastikan 'kaleido' sudah ter-install: pip install kaleido


# slide 2

In [26]:
!pip install vega_datasets


In [31]:
import pandas as pd
import altair as alt
from vega_datasets import data

# --- 1. Define File Names ---
file_suicide = 'suicide-rate-by-country-2025.csv'
print("--- Memulai script visualisasi Peta Dunia (Perbaikan) ---")

# --- 2. Load Data Peta ---
# Memuat data geografi (outline negara)
# Ini adalah data TopoJSON yang punya 'properties.name' (nama negara)
countries_geom = alt.topo_feature(data.world_110m.url, 'countries')
print("Data geografi peta dimuat.")

# --- 3. Load & Process Data Suicide ---
try:
    df_suicide_raw = pd.read_csv(file_suicide)
    print("File suicide rate berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat file: {e}")
    exit()

# Proses data suicide (ambil data 2021)
suicide_cols_to_melt = ['SuicideRateCountries_2021']
if suicide_cols_to_melt[0] not in df_suicide_raw.columns:
    print("Error: Kolom 'SuicideRateCountries_2021' tidak ditemukan.")
    exit()

df_suicide_melted = df_suicide_raw.melt(id_vars=['country'],
                                        value_vars=suicide_cols_to_melt,
                                        var_name='Suicide_Indicator',
                                        value_name='Suicide Rate')
df_suicide_melted['Suicide Rate'] = pd.to_numeric(df_suicide_melted['Suicide Rate'], errors='coerce')
# Penting: Hapus baris yang datanya NaN SETELAH konversi
df_suicide_2021 = df_suicide_melted.dropna(subset=['Suicide Rate'])
print("Data suicide 2021 selesai diproses.")


# --- 4. (DIHAPUS) ---
# Tidak perlu merge manual. Kita akan pakai transform_lookup.


# --- 5. Create Map Chart (Static Version) - DIPERBAIKI ---
try:
    chart_map = alt.Chart(countries_geom).mark_geoshape(
        stroke='black', # Garis batas antar negara
        strokeWidth=0.2
    ).project(
        type='equirectangular' # Tipe proyeksi peta
    ).encode(
        # Warnai negara berdasarkan 'Suicide Rate'
        # Gunakan alt.condition (c kecil)
        color=alt.condition(
            "!isValid(datum['Suicide Rate'])",  # Tes: jika datanya tidak valid (null)
            alt.value('#lightgray'),            # Nilai jika benar: abu-abu muda
            
            # Nilai jika salah (jika datanya ada):
            alt.Color('Suicide Rate:Q',
                      scale=alt.Scale(range='heatmap', domain=[0, 30]),
                      legend=alt.Legend(title="Suicide Rate (per 100k)")
                     )
        ),
        tooltip=[
            alt.Tooltip('country:N', title='Country'),
            alt.Tooltip('Suicide Rate:Q', title='Suicide Rate', format='.2f')
        ]
    ).transform_lookup(
        lookup='properties.name', # Kunci di PETA (nama negara standar)
        from_=alt.LookupData(df_suicide_2021, 'country', ['Suicide Rate', 'country'])
    ).properties(
        title='Global Suicide Rate (2021)',
        width=800,
        height=450
    ).interactive() # Dibuat interaktif dengan tooltip

    # Simpan sebagai file JSON (sekarang interaktif)
    chart_file = 'slide2_map_INTERACTIVE.json'
    chart_map.save(chart_file)
    
    print(f"\n--- BERHASIL ---")
    print(f"Peta dunia interaktif berhasil disimpan ke: {chart_file}")

except Exception as e:
    print(f"Error membuat Peta: {e}")

--- Memulai script visualisasi Peta Dunia (Perbaikan) ---
Data geografi peta dimuat.
File suicide rate berhasil dimuat.
Data suicide 2021 selesai diproses.

--- BERHASIL ---
Peta dunia interaktif berhasil disimpan ke: slide2_map_INTERACTIVE.json


In [32]:
# Jalankan ini di komputermu (bukan di sini)
from vega_datasets import data
world_ids = data.world_110m()
print(set(world_ids['name']))

KeyError: 'name'

In [33]:
import pandas as pd
import altair as alt
from vega_datasets import data

# --- 1. Define File Names ---
file_suicide = 'suicide-rate-by-country-2025.csv'
print("--- Memulai script visualisasi Peta Dunia (dengan Terjemahan) ---")

# --- 2. Load Data Peta ---
countries_geom = alt.topo_feature(data.world_110m.url, 'countries')
print("Data geografi peta dimuat.")

# --- 3. Load & Process Data Suicide ---
try:
    df_suicide_raw = pd.read_csv(file_suicide)
    print("File suicide rate berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat file: {e}")
    exit()

# Proses data suicide (ambil data 2021)
suicide_cols_to_melt = ['SuicideRateCountries_2021']
if suicide_cols_to_melt[0] not in df_suicide_raw.columns:
    print("Error: Kolom 'SuicideRateCountries_2021' tidak ditemukan.")
    exit()

df_suicide_melted = df_suicide_raw.melt(id_vars=['country'],
                                        value_vars=suicide_cols_to_melt,
                                        var_name='Suicide_Indicator',
                                        value_name='Suicide Rate')
df_suicide_melted['Suicide Rate'] = pd.to_numeric(df_suicide_melted['Suicide Rate'], errors='coerce')
df_suicide_2021 = df_suicide_melted.dropna(subset=['Suicide Rate']).copy()
print("Data suicide 2021 selesai diproses.")


# --- 4. MEMPERBAIKI NAMA NEGARA (BAGIAN BARU) ---
print("Menerjemahkan nama negara agar sesuai dengan peta...")

# Kamus untuk menerjemahkan nama di file-mu ke nama standar peta
translation_dict = {
    # NAMA DI FILE-MU  :  NAMA STANDAR DI PETA
    'United States': 'United States of America',
    'South Korea': 'Republic of Korea',
    'North Korea': "Dem. People's Republic of Korea",
    'Russia': 'Russian Federation',
    'Congo (Kinshasa)': 'Democratic Republic of the Congo',
    'Congo (Brazzaville)': 'Republic of the Congo',
    'Czechia': 'Czech Republic',
    'Vietnam': 'Viet Nam',
    'Laos': "Lao People's Democratic Republic",
    'Syria': 'Syrian Arab Republic',
    'Brunei': 'Brunei Darussalam',
    'Taiwan': 'Taiwan, Province of China',
    'Serbia': 'Republic of Serbia',
    'Macedonia': 'North Macedonia'
    # Tambahkan lebih banyak terjemahan di sini jika perlu
}

# Buat kolom baru dengan nama yang sudah diterjemahkan
# .replace() akan mengganti nama yang ada di kamus,
# dan membiarkan nama yang lain (seperti "Australia") apa adanya.
df_suicide_2021['country_translated'] = df_suicide_2021['country'].replace(translation_dict)
print("Penerjemahan selesai.")


# --- 5. Create Map Chart (Menggunakan Nama yang Diterjemahkan) ---
try:
    chart_map = alt.Chart(countries_geom).mark_geoshape(
        stroke='black',
        strokeWidth=0.2
    ).project(
        type='equirectangular'
    ).encode(
        # Warnai peta
        color=alt.condition(
            "!isValid(datum['Suicide Rate'])",
            alt.value('#lightgray'),
            alt.Color('Suicide Rate:Q',
                      scale=alt.Scale(range='heatmap', domain=[0, 30]),
                      legend=alt.Legend(title="Suicide Rate (per 100k)")
                     )
        ),
        # Tooltip akan menampilkan nama asli dari file-mu
        tooltip=[
            alt.Tooltip('country:N', title='Country'),
            alt.Tooltip('Suicide Rate:Q', title='Suicide Rate', format='.2f')
        ]
    ).transform_lookup(
        # Cocokkan 'properties.name' di peta...
        lookup='properties.name', 
        # ...dengan 'country_translated' di data kita
        from_=alt.LookupData(df_suicide_2021, 'country_translated', ['Suicide Rate', 'country'])
    ).properties(
        title='Global Suicide Rate (2021)',
        width=800,
        height=450
    ).interactive()

    # Simpan sebagai file JSON
    chart_file = 'slide2_map_TRANSLATED.json'
    chart_map.save(chart_file)
    
    print(f"\n--- BERHASIL ---")
    print(f"Peta dunia (yang sudah diterjemahkan) berhasil disimpan ke: {chart_file}")

except Exception as e:
    print(f"Error membuat Peta: {e}")

--- Memulai script visualisasi Peta Dunia (dengan Terjemahan) ---
Data geografi peta dimuat.
File suicide rate berhasil dimuat.
Data suicide 2021 selesai diproses.
Menerjemahkan nama negara agar sesuai dengan peta...
Penerjemahan selesai.

--- BERHASIL ---
Peta dunia (yang sudah diterjemahkan) berhasil disimpan ke: slide2_map_TRANSLATED.json


In [34]:
import pandas as pd
import altair as alt
from vega_datasets import data

# --- 1. Define File Names ---
file_suicide = 'suicide-rate-by-country-2025.csv'
print("--- Memulai script Peta Time Series (dengan Slider) ---")

# --- 2. Load Data Peta ---
countries_geom = alt.topo_feature(data.world_110m.url, 'countries')
print("Data geografi peta dimuat.")

# --- 3. Load & Process (Melt) Data Suicide ---
try:
    df_suicide_raw = pd.read_csv(file_suicide)
    print("File suicide rate berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat file: {e}")
    exit()

# Kolom tahun yang akan di-Melt
suicide_cols_to_melt = [
    'SuicideRateCountries_2019',
    'SuicideRateCountries_2020',
    'SuicideRateCountries_2021'
]
# Cek apakah kolomnya ada
existing_cols = [col for col in suicide_cols_to_melt if col in df_suicide_raw.columns]
if not existing_cols:
    print(f"Error: Tidak ditemukan kolom data suicide (seperti 'SuicideRateCountries_2019')")
    exit()

# Proses MELT
df_suicide_long = df_suicide_raw.melt(
    id_vars=['country'],
    value_vars=existing_cols,
    var_name='Indicator',
    value_name='Suicide Rate'
)

# Ekstrak Tahun dari nama kolom
df_suicide_long['Year'] = df_suicide_long['Indicator'].str.extract(r'(\d{4})').astype(int)
df_suicide_long['Suicide Rate'] = pd.to_numeric(df_suicide_long['Suicide Rate'], errors='coerce')
df_suicide_final = df_suicide_long.dropna(subset=['Suicide Rate']).copy()
print("Data suicide berhasil di-MELT ke format 'panjang'.")


# --- 4. Menerjemahkan Nama Negara ---
print("Menerjemahkan nama negara...")
translation_dict = {
    'United States': 'United States of America',
    'South Korea': 'Republic of Korea',
    'North Korea': "Dem. People's Republic of Korea",
    'Russia': 'Russian Federation',
    'Congo (Kinshasa)': 'Democratic Republic of the Congo',
    'Congo (Brazzaville)': 'Republic of the Congo',
    'Czechia': 'Czech Republic',
    'Vietnam': 'Viet Nam',
    'Laos': "Lao People's Democratic Republic",
    'Syria': 'Syrian Arab Republic',
    'Brunei': 'Brunei Darussalam',
    'Taiwan': 'Taiwan, Province of China',
    'Serbia': 'Republic of Serbia',
    'Macedonia': 'North Macedonia'
}
df_suicide_final['country_translated'] = df_suicide_final['country'].replace(translation_dict)
print("Penerjemahan selesai.")


# --- 5. Membuat Peta dengan Slider Tahun ---
try:
    # Buat Slider untuk Tahun
    # Ambil tahun min dan max dari data
    year_min = df_suicide_final['Year'].min()
    year_max = df_suicide_final['Year'].max()
    
    slider = alt.binding_range(min=year_min, max=year_max, step=1, name='Year: ')
    select_year = alt.selection_point(name="YearSelection", fields=['Year'], bind=slider, value=year_max)

    # Buat Peta
    chart_map = alt.Chart(countries_geom).mark_geoshape(
        stroke='black',
        strokeWidth=0.2
    ).project(
        type='equirectangular'
    ).encode(
        # Warnai peta
        color=alt.condition(
            "!isValid(datum['Suicide Rate'])",
            alt.value('#lightgray'),
            alt.Color('Suicide Rate:Q',
                      scale=alt.Scale(range='heatmap', domain=[0, 30]),
                      legend=alt.Legend(title="Suicide Rate (per 100k)")
                     )
        ),
        tooltip=[
            alt.Tooltip('country:N', title='Country'),
            alt.Tooltip('Year:O', title='Year'),
            alt.Tooltip('Suicide Rate:Q', title='Suicide Rate', format='.2f')
        ]
    ).add_params(
        select_year  # Tambahkan slider ke chart
    ).transform_filter(
        select_year  # Filter data peta berdasarkan tahun di slider
    ).transform_lookup(
        lookup='properties.name', 
        from_=alt.LookupData(df_suicide_final, 'country_translated', ['Suicide Rate', 'country', 'Year'])
    ).properties(
        title='Global Suicide Rate by Year',
        width=800,
        height=450
    )

    # Simpan sebagai file JSON
    chart_file = 'slide2_map_TIMESERIES.json'
    chart_map.save(chart_file)
    
    print(f"\n--- BERHASIL ---")
    print(f"Peta dunia INTERAKTIF (dengan slider tahun) berhasil disimpan ke: {chart_file}")

except Exception as e:
    print(f"Error membuat Peta: {e}")

--- Memulai script Peta Time Series (dengan Slider) ---
Data geografi peta dimuat.
File suicide rate berhasil dimuat.
Data suicide berhasil di-MELT ke format 'panjang'.
Menerjemahkan nama negara...
Penerjemahan selesai.

--- BERHASIL ---
Peta dunia INTERAKTIF (dengan slider tahun) berhasil disimpan ke: slide2_map_TIMESERIES.json


In [36]:
import pandas as pd
from vega_datasets import data

# 'data.countries()' memuat tabel data negara yang dipakai peta
try:
    df_map_names = data.countries()
    print("--- NAMA KOLOM YANG ADA DI DATA PETA ---")
    print(df_map_names.columns)
    
except Exception as e:
    print(f"Terjadi error: {e}")

--- NAMA KOLOM YANG ADA DI DATA PETA ---
Index(['_comment', 'year', 'fertility', 'life_expect', 'n_fertility',
       'n_life_expect', 'country', 'p_fertility', 'p_life_expect'],
      dtype='object')


In [37]:
import pandas as pd
from vega_datasets import data

# 'data.countries()' memuat tabel data negara yang dipakai peta
try:
    df_map_names = data.countries()
    
    # Ambil semua nama unik dari kolom 'country'
    standard_names = sorted(list(set(df_map_names['country'])))
    
    print("--- DAFTAR NAMA STANDAR DI PETA ---")
    print(standard_names)
    print("\nTotal nama standar:", len(standard_names))
    
except Exception as e:
    print(f"Terjadi error: {e}")

--- DAFTAR NAMA STANDAR DI PETA ---
['Afghanistan', 'Argentina', 'Aruba', 'Australia', 'Austria', 'Bahamas', 'Bangladesh', 'Barbados', 'Belgium', 'Bolivia', 'Brazil', 'Canada', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Finland', 'France', 'Georgia', 'Germany', 'Greece', 'Grenada', 'Haiti', 'Hong Kong', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Kenya', 'Lebanon', 'Mexico', 'Netherlands', 'New Zealand', 'Nigeria', 'North Korea', 'Norway', 'Pakistan', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Rwanda', 'Saudi Arabia', 'South Africa', 'South Korea', 'Spain', 'Switzerland', 'Turkey', 'United Kingdom', 'United States', 'Venezuela']

Total nama standar: 63


In [38]:
import pandas as pd
import altair as alt
from vega_datasets import data

# --- 1. Define File Names ---
file_suicide = 'suicide-rate-by-country-2025.csv'
print("--- Memulai script Peta Time Series (Kamus Final) ---")

# --- 2. Load Data Peta ---
countries_geom = alt.topo_feature(data.world_110m.url, 'countries')
print("Data geografi peta dimuat.")

# --- 3. Load & Process (Melt) Data Suicide ---
try:
    df_suicide_raw = pd.read_csv(file_suicide)
    print("File suicide rate berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat file: {e}")
    exit()

# Kolom tahun yang akan di-Melt
suicide_cols_to_melt = [
    'SuicideRateCountries_2019',
    'SuicideRateCountries_2020',
    'SuicideRateCountries_2021'
]
existing_cols = [col for col in suicide_cols_to_melt if col in df_suicide_raw.columns]
if not existing_cols:
    print(f"Error: Tidak ditemukan kolom data suicide (seperti 'SuicideRateCountries_2019')")
    exit()

# Proses MELT
df_suicide_long = df_suicide_raw.melt(
    id_vars=['country'],
    value_vars=existing_cols,
    var_name='Indicator',
    value_name='Suicide Rate'
)
df_suicide_long['Year'] = df_suicide_long['Indicator'].str.extract(r'(\d{4})').astype(int)
df_suicide_long['Suicide Rate'] = pd.to_numeric(df_suicide_long['Suicide Rate'], errors='coerce')
df_suicide_final = df_suicide_long.dropna(subset=['Suicide Rate']).copy()
print("Data suicide berhasil di-MELT.")


# --- 4. Menerjemahkan Nama Negara (KAMUS YANG BENAR) ---
print("Menerjemahkan nama negara agar sesuai dengan Peta Standar...")
# Ini adalah kamus yang mencocokkan nama umum
# ke nama standar di 'properties.name' file peta 'world_110m'
translation_dict = {
    'United States': 'United States of America',
    'South Korea': 'Republic of Korea',
    'North Korea': "Dem. People's Republic of Korea",
    'Russia': 'Russian Federation',
    'Congo (Kinshasa)': 'Democratic Republic of the Congo',
    'Congo (Brazzaville)': 'Republic of the Congo',
    'Czechia': 'Czech Republic',
    'Vietnam': 'Viet Nam',
    'Laos': "Lao People's Democratic Republic",
    'Syria': 'Syrian Arab Republic',
    'Brunei': 'Brunei Darussalam',
    'Taiwan': 'Taiwan', # Peta standar sering hanya menyebut 'Taiwan'
    'Serbia': 'Republic of Serbia',
    'Macedonia': 'North Macedonia',
    'Bosnia and Herzegovina': 'Bosnia and Herz.',
    'Dominican Republic': 'Dominican Rep.',
    'Central African Republic': 'Central African Rep.',
    'Equatorial Guinea': 'Eq. Guinea',
    'Eswatini': 'eSwatini',
    'South Sudan': 'S. Sudan',
    'Western Sahara': 'W. Sahara'
}

df_suicide_final['country_translated'] = df_suicide_final['country'].replace(translation_dict)
print("Penerjemahan selesai.")


# --- 5. Membuat Peta dengan Slider Tahun ---
try:
    year_min = df_suicide_final['Year'].min()
    year_max = df_suicide_final['Year'].max()
    
    slider = alt.binding_range(min=year_min, max=year_max, step=1, name='Year: ')
    select_year = alt.selection_point(name="YearSelection", fields=['Year'], bind=slider, value=year_max)

    chart_map = alt.Chart(countries_geom).mark_geoshape(
        stroke='black',
        strokeWidth=0.2
    ).project(
        type='equirectangular'
    ).encode(
        color=alt.condition(
            "!isValid(datum['Suicide Rate'])",
            alt.value('#lightgray'),
            alt.Color('Suicide Rate:Q',
                      scale=alt.Scale(range='heatmap', domain=[0, 30]),
                      legend=alt.Legend(title="Suicide Rate (per 100k)")
                     )
        ),
        tooltip=[
            alt.Tooltip('country:N', title='Country (from your file)'),
            alt.Tooltip('Year:O', title='Year'),
            alt.Tooltip('Suicide Rate:Q', title='Suicide Rate', format='.2f')
        ]
    ).add_params(
        select_year
    ).transform_filter(
        select_year
    ).transform_lookup(
        # Cocokkan 'properties.name' di peta...
        lookup='properties.name', 
        # ...dengan 'country_translated' (nama yang sudah diterjemahkan)
        from_=alt.LookupData(df_suicide_final, 'country_translated', ['Suicide Rate', 'country', 'Year'])
    ).properties(
        title='Global Suicide Rate by Year',
        width=800,
        height=450
    ).interactive()

    # Simpan sebagai file JSON
    chart_file = 'slide2_map_TIMESERIES_FINAL.json'
    chart_map.save(chart_file)
    
    print(f"\n--- BERHASIL ---")
    print(f"Peta dunia INTERAKTIF (Final) berhasil disimpan ke: {chart_file}")

except Exception as e:
    print(f"Error membuat Peta: {e}")

--- Memulai script Peta Time Series (Kamus Final) ---
Data geografi peta dimuat.
File suicide rate berhasil dimuat.
Data suicide berhasil di-MELT.
Menerjemahkan nama negara agar sesuai dengan Peta Standar...
Penerjemahan selesai.

--- BERHASIL ---
Peta dunia INTERAKTIF (Final) berhasil disimpan ke: slide2_map_TIMESERIES_FINAL.json


In [39]:
import pandas as pd
import altair as alt
from vega_datasets import data

# --- 1. Define File Names ---
file_suicide = 'suicide-rate-by-country-2025.csv'
print("--- Memulai script Peta (Tanpa Kamus Terjemahan) ---")

# --- 2. Load Data Peta ---
countries_geom = alt.topo_feature(data.world_110m.url, 'countries')
print("Data geografi peta dimuat.")

# --- 3. Load & Process (Melt) Data Suicide ---
try:
    df_suicide_raw = pd.read_csv(file_suicide)
    print("File suicide rate berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat file: {e}")
    exit()

# Kolom tahun yang akan di-Melt
suicide_cols_to_melt = [
    'SuicideRateCountries_2019',
    'SuicideRateCountries_2020',
    'SuicideRateCountries_2021'
]
existing_cols = [col for col in suicide_cols_to_melt if col in df_suicide_raw.columns]
if not existing_cols:
    print(f"Error: Tidak ditemukan kolom data suicide")
    exit()

# Proses MELT
df_suicide_long = df_suicide_raw.melt(
    id_vars=['country'],
    value_vars=existing_cols,
    var_name='Indicator',
    value_name='Suicide Rate'
)
df_suicide_long['Year'] = df_suicide_long['Indicator'].str.extract(r'(\d{4})').astype(int)
df_suicide_long['Suicide Rate'] = pd.to_numeric(df_suicide_long['Suicide Rate'], errors='coerce')
df_suicide_final = df_suicide_long.dropna(subset=['Suicide Rate']).copy()
print("Data suicide berhasil di-MELT.")


# --- 4. (LANGKAH PENERJEMAHAN DIHAPUS) ---
print("Tidak ada penerjemahan. Mencocokkan nama 'country' langsung...")


# --- 5. Membuat Peta dengan Slider Tahun ---
try:
    year_min = df_suicide_final['Year'].min()
    year_max = df_suicide_final['Year'].max()
    
    slider = alt.binding_range(min=year_min, max=year_max, step=1, name='Year: ')
    select_year = alt.selection_point(name="YearSelection", fields=['Year'], bind=slider, value=year_max)

    chart_map = alt.Chart(countries_geom).mark_geoshape(
        stroke='black',
        strokeWidth=0.2
    ).project(
        type='equirectangular'
    ).encode(
        color=alt.condition(
            "!isValid(datum['Suicide Rate'])",
            alt.value('#lightgray'),
            alt.Color('Suicide Rate:Q',
                      scale=alt.Scale(range='heatmap', domain=[0, 30]),
                      legend=alt.Legend(title="Suicide Rate (per 100k)")
                     )
        ),
        tooltip=[
            alt.Tooltip('country:N', title='Country (from your file)'),
            alt.Tooltip('Year:O', title='Year'),
            alt.Tooltip('Suicide Rate:Q', title='Suicide Rate', format='.2f')
        ]
    ).add_params(
        select_year
    ).transform_filter(
        select_year
    ).transform_lookup(
        # Cocokkan 'properties.name' di peta...
        lookup='properties.name', 
        # ...dengan 'country' (nama ASLI dari file-mu)
        from_=alt.LookupData(df_suicide_final, 'country', ['Suicide Rate', 'country', 'Year'])
    ).properties(
        title='Global Suicide Rate by Year',
        width=800,
        height=450
    ).interactive()

    # Simpan sebagai file JSON
    chart_file = 'slide2_map_NO_TRANSLATION.json'
    chart_map.save(chart_file)
    
    print(f"\n--- BERHASIL ---")
    print(f"Peta dunia (Tanpa Terjemahan) berhasil disimpan ke: {chart_file}")

except Exception as e:
    print(f"Error membuat Peta: {e}")

--- Memulai script Peta (Tanpa Kamus Terjemahan) ---
Data geografi peta dimuat.
File suicide rate berhasil dimuat.
Data suicide berhasil di-MELT.
Tidak ada penerjemahan. Mencocokkan nama 'country' langsung...

--- BERHASIL ---
Peta dunia (Tanpa Terjemahan) berhasil disimpan ke: slide2_map_NO_TRANSLATION.json


In [41]:
pip install plotly

  Using cached plotly-6.3.1-py3-none-any.whl.metadata (8.5 kB)
Using cached plotly-6.3.1-py3-none-any.whl (9.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [42]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Mengatur 'template' default agar background-nya putih
pio.templates.default = "plotly_white"

print("--- Memulai script Peta (Cara Baru: Plotly) ---")

# --- 1. Define File Names ---
file_suicide = 'suicide-rate-by-country-2025.csv'

# --- 2. Load & Process (Melt) Data Suicide ---
try:
    df_suicide_raw = pd.read_csv(file_suicide)
    print("File suicide rate berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat file: {e}")
    exit()

# Kolom tahun yang akan di-Melt
suicide_cols_to_melt = [
    'SuicideRateCountries_2019',
    'SuicideRateCountries_2020',
    'SuicideRateCountries_2021'
]
existing_cols = [col for col in suicide_cols_to_melt if col in df_suicide_raw.columns]
if not existing_cols:
    print(f"Error: Tidak ditemukan kolom data suicide")
    exit()

# Proses MELT
df_suicide_long = df_suicide_raw.melt(
    id_vars=['country'],
    value_vars=existing_cols,
    var_name='Indicator',
    value_name='Suicide Rate'
)
df_suicide_long['Year'] = df_suicide_long['Indicator'].str.extract(r'(\d{4})').astype(int)
df_suicide_long['Suicide Rate'] = pd.to_numeric(df_suicide_long['Suicide Rate'], errors='coerce')
df_suicide_final = df_suicide_long.dropna(subset=['Suicide Rate']).copy()
print("Data suicide berhasil di-MELT.")


# --- 3. Membuat Peta Plotly ---
try:
    print("Membuat peta Plotly...")
    fig = px.choropleth(
        df_suicide_final,
        locations="country",            # Kolom di datamu yang berisi nama negara
        locationmode="country names",   # Beri tahu Plotly untuk mencocokkan nama
        color="Suicide Rate",           # Kolom untuk data warna
        hover_name="country",           # Apa yang muncul saat di-hover
        animation_frame="Year",         # Ini yang membuat SLIDER TAHUN
        color_continuous_scale=px.colors.sequential.YlOrRd, # Skala warna (kuning ke merah)
        projection="natural earth",     # Tipe proyeksi peta
        title="Global Suicide Rate by Year"
    )
    
    # Memperbarui layout legenda
    fig.update_layout(
        coloraxis_colorbar=dict(
            title="Suicide Rate<br>(per 100k)"
        )
    )
    
    # --- 4. Menyimpan Peta ---
    
    # Cara 1: Simpan sebagai HTML (Interaktif, Pasti Berhasil)
    html_file = 'slide2_map_plotly.html'
    fig.write_html(html_file)
    print(f"\n--- BERHASIL (HTML) ---")
    print(f"Peta INTERAKTIF disimpan ke: {html_file}")
    print("Kamu bisa buka file ini di browser (Chrome, Firefox, dll.)")

    # Cara 2: Coba Simpan sebagai PNG (Statis)
    # Ini butuh library 'kaleido'. Mungkin gagal.
    try:
        png_file = 'slide2_map_plotly.png'
        # engine='kaleido' adalah default, tapi kita tulis agar jelas
        fig.write_image(png_file, width=1000, height=500, engine='kaleido')
        print(f"\n--- BERHASIL (PNG) ---")
        print(f"Gambar peta (PNG) berhasil disimpan ke: {png_file}")
    except ImportError:
        print("\n--- INFO (PNG Gagal) ---")
        print("Gagal menyimpan PNG karena library 'kaleido' tidak ter-install di environment ini.")
        print("Tapi jangan khawatir, file HTML-nya sudah 100% berhasil dibuat.")
    except Exception as e_png:
        print(f"\nError saat menyimpan PNG: {e_png}")
        
except Exception as e_fig:
    print(f"Error membuat Peta Plotly: {e_fig}")

--- Memulai script Peta (Cara Baru: Plotly) ---
File suicide rate berhasil dimuat.
Data suicide berhasil di-MELT.
Membuat peta Plotly...


C:\Users\DARA OKTAVIANA\AppData\Local\Temp\ipykernel_23472\2929094867.py:48: DeprecationWarning: The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.
  fig = px.choropleth(



--- BERHASIL (HTML) ---
Peta INTERAKTIF disimpan ke: slide2_map_plotly.html
Kamu bisa buka file ini di browser (Chrome, Firefox, dll.)

Error saat menyimpan PNG: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido



C:\Users\DARA OKTAVIANA\AppData\Local\Temp\ipykernel_23472\2929094867.py:81: DeprecationWarning:


Support for the 'engine' argument is deprecated and will be removed after September 2025.
Kaleido will be the only supported engine at that time.




# 3. DATA GLOBAL 3 TAHUN

In [43]:
import pandas as pd
import altair as alt

# --- 1. Tentukan Nama File ---
# Ini adalah file gabungan "master" yang sudah kamu buat
file_gabungan = 'dataset_gabungan_final.csv'

print("--- Memulai script Plot 1: Tren Global 3 Tahun ---")

# --- 2. Muat Data Gabungan ---
try:
    df_gabungan = pd.read_csv(file_gabungan)
    print(f"Berhasil memuat {file_gabungan}")
except FileNotFoundError:
    print(f"Error: File '{file_gabungan}' tidak ditemukan.")
    print("Pastikan file itu ada di folder yang sama dengan script ini.")
    exit()
except Exception as e:
    print(f"Error saat memuat file: {e}")
    exit()

# --- 3. Proses Data ---
# Kita hanya butuh kolom 'Year' dan 'Suicide Rate'
if 'Year' not in df_gabungan.columns or 'Suicide Rate' not in df_gabungan.columns:
    print("Error: Kolom 'Year' atau 'Suicide Rate' tidak ditemukan di file.")
    exit()

# Hitung rata-rata global suicide rate untuk setiap tahun
# 'groupby('Year')' -> Kelompokkan berdasarkan 2019, 2020, 2021
# 'mean()' -> Ambil rata-ratanya
# 'reset_index()' -> Ubah kembali jadi DataFrame yang rapi
df_tren = df_gabungan.groupby('Year')['Suicide Rate'].mean().reset_index()

print("Data rata-rata global per tahun:")
print(df_tren.head())

# --- 4. Buat Visualisasi (Line Chart) ---
try:
    chart = alt.Chart(df_tren).mark_line(
        point=True,  # Tampilkan titik di setiap tahun
        strokeWidth=3 # Tebalkan garis
    ).encode(
        # Sumbu X: Tahun. 'O' = Ordinal (kategorikal), bukan angka
        x=alt.X('Year:O', axis=alt.Axis(title='Year', labelAngle=0)),
        
        # Sumbu Y: Rata-rata Suicide Rate
        y=alt.Y('Suicide Rate', axis=alt.Axis(title='Average Global Suicide Rate (per 100k)')),
        
        # Tooltip untuk interaktivitas
        tooltip=[
            alt.Tooltip('Year:O'),
            alt.Tooltip('Suicide Rate', title='Average Rate', format='.2f') # Format 2 angka desimal
        ]
    ).properties(
        title='Global Suicide Rate Trend (2019-2021)',
        width=500
    ).interactive()

    # Simpan sebagai file JSON
    chart_file = 'slide3_global_trend.json'
    chart.save(chart_file)
    
    print(f"\n--- BERHASIL ---")
    print(f"Grafik Tren Global berhasil disimpan ke: {chart_file}")

except Exception as e:
    print(f"Error saat membuat chart: {e}")

--- Memulai script Plot 1: Tren Global 3 Tahun ---
Berhasil memuat dataset_gabungan_final.csv
Data rata-rata global per tahun:
   Year  Suicide Rate
0  2019      9.440110
1  2020     13.675419
2  2021      9.510985

--- BERHASIL ---
Grafik Tren Global berhasil disimpan ke: slide3_global_trend.json


# (Plot 2: Age vs Suicide - Per Tahun)

In [ ]:
import pandas as pd
import altair as alt

# --- 1. Tentukan Nama File ---
file_gabungan = 'dataset_gabungan_final.csv'

print("--- Memulai script Plot 2: Age vs Suicide (Faceted by Year) ---")

# --- 2. Muat Data Gabungan ---
try:
    df_gabungan = pd.read_csv(file_gabungan)
    print(f"Berhasil memuat {file_gabungan}")
except FileNotFoundError:
    print(f"Error: File '{file_gabungan}' tidak ditemukan.")
    exit()
except Exception as e:
    print(f"Error saat memuat file: {e}")
    exit()

# --- 3. Proses Data ---
col_age = 'SP.POP.65UP.TO.ZS' # Kolom populasi 65+
col_rate = 'Suicide Rate'
col_year = 'Year'
col_country = 'Country Name' # Untuk tooltip

required_cols = [col_age, col_rate, col_year, col_country]
missing_cols = [col for col in required_cols if col not in df_gabungan.columns]
if missing_cols:
    print(f"Error: Kolom berikut tidak ditemukan: {', '.join(missing_cols)}")
    exit()

# Ambil data untuk 3 tahun
years_to_plot = [2019, 2020, 2021]
df_filtered = df_gabungan[df_gabungan[col_year].isin(years_to_plot)].copy()

# Bersihkan data, hapus baris yang datanya kosong untuk plot ini
df_filtered = df_filtered.dropna(subset=[col_age, col_rate])

if df_filtered.empty:
    print("Error: Tidak ada data valid untuk tahun 2019-2021 setelah dibersihkan.")
    exit()

print(f"Data 2019-2021 siap untuk di-plot (total {len(df_filtered)} data poin).")

# --- 4. Buat Visualisasi (Faceted Scatter Plot) ---
try:
    # Chart dasar
    base = alt.Chart(df_filtered).encode(
        x=alt.X(col_age, axis=alt.Axis(title='Population Ages 65+ (% of Total)')),
        y=alt.Y(col_rate, axis=alt.Axis(title='Suicide Rate (per 100k)'))
    )
    
    # Layer 1: Titik-titik scatter plot
    scatter_points = base.mark_point(opacity=0.7, filled=True).encode(
        tooltip=[
            alt.Tooltip(col_country, title='Country'),
            alt.Tooltip(col_year, title='Year'),
            alt.Tooltip(col_age, title='Pop. 65+ (%)', format='.1f'),
            alt.Tooltip(col_rate, title='Suicide Rate', format='.1f')
        ]
    )
    
    # Layer 2: Garis regresi (tren)
    regression_line = base.transform_regression(
        col_age, col_rate, method='linear'
    ).mark_line(
        color='red',
        strokeWidth=2
    )

    # Gabungkan kedua layer
    chart = (scatter_points + regression_line).properties(
        title='Suicide Rate vs. Elderly Population (% 65+)',
        width=250 # Buat tiap plot lebih kecil agar muat 3
    ).interactive()

    # --- INI BAGIAN KUNCINYA ---
    # Susun chart berdasarkan kolom 'Year'
    faceted_chart = chart.facet(
        column=alt.Column('Year:O', header=alt.Header(titleOrient="bottom", labelOrient="bottom"))
    ).resolve_scale(
        y='shared' # Pastikan sumbu Y semua plot sama
    )

    # Simpan sebagai file JSON
    chart_file = 'slide3_age_vs_suicide_faceted.json'
    faceted_chart.save(chart_file)
    
    print(f"\n--- BERHASIL ---")
    print(f"Grafik 'Age vs Suicide' (per tahun) berhasil disimpan ke: {chart_file}")

except Exception as e:
    print(f"Error saat membuat chart: {e}")

--- Memulai script Plot 2: Age vs Suicide (Faceted by Year) ---
Berhasil memuat dataset_gabungan_final.csv
Error: Kolom berikut tidak ditemukan: SP.POP.65UP.TO.ZS


KeyError: ['SP.POP.65UP.TO.ZS']

: 

In [1]:
import pandas as pd

# --- 1. Tentukan Nama File ---
file_gabungan = 'dataset_gabungan_final.csv'

print("--- Memeriksa Kolom di 'dataset_gabungan_final.csv' ---")

# --- 2. Muat Data dan Tampilkan Kolom ---
try:
    df_gabungan = pd.read_csv(file_gabungan)
    print("File berhasil dimuat.")
    print("\n--- DAFTAR KOLOM YANG DITEMUKAN ---")
    print(df_gabungan.columns.tolist())
    
    # Coba cari kolom yang mirip
    potential_cols = [col for col in df_gabungan.columns if 'POP' in col or '65' in col]
    if potential_cols:
        print(f"\nKolom yang mungkin berhubungan dengan 'Populasi': {potential_cols}")
    else:
        print("\nTidak ditemukan kolom yang mengandung 'POP' or '65'.")
        
except FileNotFoundError:
    print(f"Error: File '{file_gabungan}' tidak ditemukan.")
except Exception as e:
    print(f"Error saat memuat file: {e}")

--- Memeriksa Kolom di 'dataset_gabungan_final.csv' ---
File berhasil dimuat.

--- DAFTAR KOLOM YANG DITEMUKAN ---
['Country Name', 'Year', 'SP.POP.0014.TO.ZS', 'SP.POP.1564.TO.ZS', 'HDI', 'Suicide Rate']

Kolom yang mungkin berhubungan dengan 'Populasi': ['SP.POP.0014.TO.ZS', 'SP.POP.1564.TO.ZS']


In [1]:
import pandas as pd

# --- 1. TENTUKAN NAMA FILE ---
# !!! GANTI INI DENGAN NAMA FILE-MU YANG SEBENARNYA !!!
file_wdi = 'WDICSV-rev.csv' 
file_hdi = 'human-development-index-(hdi)-by-country-2025.csv'
file_suicide = 'suicide-rate-by-country-2025.csv'

# Indikator WDI yang ingin kamu ambil
wdi_indicators_to_keep = [
    'SP.POP.0014.TO.ZS', # Populasi 0-14
    'SP.POP.1564.TO.ZS', # Populasi 15-64
    'SP.POP.65UP.TO.ZS', # Populasi 65+
    'SL.UEM.TOTL.ZS'     # Unemployment Total
]

# Rentang tahun utama untuk analisis
years_to_keep = [2019, 2020, 2021, 2022, 2023]

# --- 2. MEMBACA SEMUA FILE ---
try:
    df_wdi_raw = pd.read_csv(file_wdi)
    df_hdi_raw = pd.read_csv(file_hdi)
    df_suicide_raw = pd.read_csv(file_suicide)
except FileNotFoundError as e:
    print(f"Error: File tidak ditemukan. Pastikan nama file sudah benar.")
    print(e)
    exit()

print("Semua file berhasil dibaca.")

# --- 3. PROSES DATA WDICSV ---
print("Memproses data WDI...")
df_wdi_filtered = df_wdi_raw[df_wdi_raw['Indicator Code'].isin(wdi_indicators_to_keep)].copy()
id_vars_wdi = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']
value_vars_wdi = [col for col in df_wdi_filtered.columns if col not in id_vars_wdi]
df_wdi_melted = df_wdi_filtered.melt(id_vars=id_vars_wdi,
                                   value_vars=value_vars_wdi,
                                   var_name='Year',
                                   value_name='Value')
df_wdi_melted['Year'] = pd.to_numeric(df_wdi_melted['Year'], errors='coerce')
df_wdi_melted = df_wdi_melted.dropna(subset=['Year'])
df_wdi_melted['Year'] = df_wdi_melted['Year'].astype(int)
df_wdi_melted = df_wdi_melted[df_wdi_melted['Year'].isin(years_to_keep)]
df_wdi_processed = df_wdi_melted.pivot_table(index=['Country Name', 'Year'],
                                           columns='Indicator Code',
                                           values='Value').reset_index()
df_wdi_processed.columns.name = None
print("Data WDI selesai diproses.")

# --- 4. PROSES DATA HDI ---
print("Memproses data HDI...")
hdi_cols_to_melt = [
    'HumanDevelopmentIndex_2019',
    'HumanDevelopmentIndex_2020',
    'HumanDevelopmentIndex_2021',
    'HumanDevelopmentIndex_2022',
    'HumanDevelopmentIndex_2023'
]
hdi_cols_exist = [col for col in hdi_cols_to_melt if col in df_hdi_raw.columns]
df_hdi_melted = df_hdi_raw.melt(id_vars=['country'],
                                value_vars=hdi_cols_exist,
                                var_name='Indicator',
                                value_name='HDI')
df_hdi_melted['Year'] = df_hdi_melted['Indicator'].str.extract(r'(\d{4})').astype(int)
df_hdi_processed = df_hdi_melted.rename(columns={'country': 'Country Name'})
df_hdi_processed = df_hdi_processed[['Country Name', 'Year', 'HDI']]
print("Data HDI selesai diproses.")

# --- 5. PROSES DATA SUICIDE RATE ---
print("Memproses data Suicide Rate...")
suicide_cols_to_melt = [
    'SuicideRateCountries_2019',
    'SuicideRateCountries_2020',
    'SuicideRateCountries_2021'
]
suicide_cols_exist = [col for col in suicide_cols_to_melt if col in df_suicide_raw.columns]
df_suicide_melted = df_suicide_raw.melt(id_vars=['country'],
                                        value_vars=suicide_cols_exist,
                                        var_name='Indicator',
                                        value_name='Suicide Rate')
df_suicide_melted['Year'] = df_suicide_melted['Indicator'].str.extract(r'(\d{4})')
df_suicide_melted['Year'] = pd.to_numeric(df_suicide_melted['Year']).astype(int)
df_suicide_processed = df_suicide_melted.rename(columns={'country': 'Country Name'})
df_suicide_processed = df_suicide_processed[['Country Name', 'Year', 'Suicide Rate']]
print("Data Suicide Rate selesai diproses.")

# --- 6. GABUNGKAN (MERGE) SEMUA DATA ---
# INI BAGIAN YANG HILANG
print("Menggabungkan semua data...")

# Kita mulai dengan data WDI
df_master = df_wdi_processed.copy()

# Gabungkan dengan data HDI
# 'how='inner'' berarti kita HANYA menyimpan baris yang ada di KEDUA tabel
# (cocok berdasarkan 'Country Name' dan 'Year')
df_master = pd.merge(df_master, df_hdi_processed, on=['Country Name', 'Year'], how='inner')

# Gabungkan dengan data Suicide
df_master = pd.merge(df_master, df_suicide_processed, on=['Country Name', 'Year'], how='inner')

print("Data berhasil digabungkan.")

# --- 7. FINALISASI & SIMPAN CSV BARU ---
# INI JUGA BAGIAN YANG HILANG

# Data kita (2019-2021) sudah bersih karena 'how=inner'
# Kita bisa langsung simpan
output_file = 'dataset_master_lengkap.csv'
df_master.to_csv(output_file, index=False)

print(f"\n--- SELESAI ---")
print(f"File master baru berhasil disimpan sebagai: {output_file}")
print("File ini sekarang berisi data WDI (termasuk 65+ dan Unemployment), HDI, dan Suicide Rate.")

print("\n5 baris pertama data gabungan:")
print(df_master.head())
print("\nInfo data:")
df_master.info()

Semua file berhasil dibaca.
Memproses data WDI...
Data WDI selesai diproses.
Memproses data HDI...
Data HDI selesai diproses.
Memproses data Suicide Rate...
Data Suicide Rate selesai diproses.
Menggabungkan semua data...
Data berhasil digabungkan.

--- SELESAI ---
File master baru berhasil disimpan sebagai: dataset_master_lengkap.csv
File ini sekarang berisi data WDI (termasuk 65+ dan Unemployment), HDI, dan Suicide Rate.

5 baris pertama data gabungan:
  Country Name  Year  SL.UEM.TOTL.ZS  SP.POP.0014.TO.ZS  SP.POP.1564.TO.ZS  \
0  Afghanistan  2019          11.185          44.576832          53.047015   
1  Afghanistan  2020          11.710          44.224104          53.405162   
2  Afghanistan  2021          11.994          43.908125          53.739325   
3      Albania  2019          11.466          17.744292          68.007610   
4      Albania  2020          11.690          17.521976          67.711020   

   SP.POP.65UP.TO.ZS    HDI  Suicide Rate  
0           2.376153  0.492  

In [ ]:
import pandas as pd
import altair as alt

# --- 1. Tentukan Nama File ---
# Ini adalah file master LENGKAP yang baru kita buat
file_master = 'dataset_master_lengkap.csv'

print("--- Memulai script Plot 2: Age vs Suicide (Faceted by Year) ---")
print(f"Menggunakan file: {file_master}")

# --- 2. Muat Data Gabungan ---
try:
    df_master = pd.read_csv(file_master)
    print(f"Berhasil memuat {file_master}")
except FileNotFoundError:
    print(f"Error: File '{file_master}' tidak ditemukan.")
    print("Pastikan kamu sudah menjalankan script sebelumnya untuk membuat file ini.")
    exit()
except Exception as e:
    print(f"Error saat memuat file: {e}")
    exit()

# --- 3. Proses Data ---
# Tentukan kolom yang kita butuhkan
col_age = 'SP.POP.65UP.TO.ZS' # Kolom populasi 65+ (YANG SEKARANG SUDAH ADA)
col_rate = 'Suicide Rate'
col_year = 'Year'
col_country = 'Country Name' # Untuk tooltip

# Cek apakah semua kolom ada
required_cols = [col_age, col_rate, col_year, col_country]
missing_cols = [col for col in required_cols if col not in df_master.columns]
if missing_cols:
    print(f"Error: Kolom berikut tidak ditemukan di {file_master}: {', '.join(missing_cols)}")
    print(f"Kolom yang ada: {df_master.columns.tolist()}")
    exit()
else:
    print("Semua kolom yang dibutuhkan (Age 65+, Suicide Rate, Year) ditemukan.")

# Ambil data untuk 3 tahun
years_to_plot = [2019, 2020, 2021]
df_filtered = df_master[df_master[col_year].isin(years_to_plot)].copy()

# Bersihkan data, hapus baris yang datanya kosong untuk plot ini
df_filtered = df_filtered.dropna(subset=[col_age, col_rate])

if df_filtered.empty:
    print("Error: Tidak ada data valid untuk tahun 2019-2021 setelah dibersihkan.")
    exit()

print(f"Data 2019-2021 siap untuk di-plot (total {len(df_filtered)} data poin).")

# --- 4. Buat Visualisasi (Faceted Scatter Plot) ---
try:
    # Chart dasar
    base = alt.Chart(df_filtered).encode(
        x=alt.X(col_age, axis=alt.Axis(title='Population Ages 65+ (% of Total)')),
        y=alt.Y(col_rate, axis=alt.Axis(title='Suicide Rate (per 100k)'))
    )
    
    # Layer 1: Titik-titik scatter plot
    scatter_points = base.mark_point(opacity=0.7, filled=True).encode(
        tooltip=[
            alt.Tooltip(col_country, title='Country'),
            alt.Tooltip(col_year, title='Year'),
            alt.Tooltip(col_age, title='Pop. 65+ (%)', format='.1f'),
            alt.Tooltip(col_rate, title='Suicide Rate', format='.1f')
        ]
    )
    
    # Layer 2: Garis regresi (tren)
    regression_line = base.transform_regression(
        col_age, col_rate, method='linear'
    ).mark_line(
        color='red',
        strokeWidth=2
    )

    # Gabungkan kedua layer
    chart = (scatter_points + regression_line).properties(
        title='Suicide Rate vs. Elderly Population (% 65+)',
        width=250 # Buat tiap plot lebih kecil agar muat 3
    ).interactive()

    # Susun chart berdasarkan kolom 'Year'
    faceted_chart = chart.facet(
        column=alt.Column('Year:O', header=alt.Header(titleOrient="bottom", labelOrient="bottom"))
    ).resolve_scale(
        y='shared' # Pastikan sumbu Y semua plot sama
    )

    # Simpan sebagai file JSON
    chart_file = 'slide3_age_vs_suicide_faceted.json'
    faceted_chart.save(chart_file)
    
    print(f"\n--- BERHASIL ---")
    print(f"Grafik 'Age vs Suicide' (per tahun) berhasil disimpan ke: {chart_file}")

except Exception as e:
    print(f"Error saat membuat chart: {e}")

In [2]:
import pandas as pd
import altair as alt

# --- 1. Tentukan Nama File ---
file_master = 'dataset_master_lengkap.csv'

print("--- Memulai script Plot 2: Age vs Suicide (Grid 3x3 Lengkap) ---")
print(f"Menggunakan file: {file_master}")

# --- 2. Muat Data Gabungan ---
try:
    df_master = pd.read_csv(file_master)
    print(f"Berhasil memuat {file_master}")
except FileNotFoundError:
    print(f"Error: File '{file_master}' tidak ditemukan.")
    exit()
except Exception as e:
    print(f"Error saat memuat file: {e}")
    exit()

# --- 3. Proses Data (MELT Kolom Usia) ---
col_rate = 'Suicide Rate'
col_year = 'Year'
col_country = 'Country Name'
age_cols = ['SP.POP.0014.TO.ZS', 'SP.POP.1564.TO.ZS', 'SP.POP.65UP.TO.ZS']

# Cek apakah semua kolom ada
required_cols = age_cols + [col_rate, col_year, col_country]
missing_cols = [col for col in required_cols if col not in df_master.columns]
if missing_cols:
    print(f"Error: Kolom berikut tidak ditemukan: {', '.join(missing_cols)}")
    exit()
else:
    print("Semua kolom (3 Kelompok Usia, Suicide Rate, Year) ditemukan.")

# 'Melt' data usia agar menjadi format "panjang"
df_melted = df_master.melt(
    id_vars=[col_country, col_year, col_rate],
    value_vars=age_cols,
    var_name='Age Group Indicator', # Ini akan berisi nama kolom (misal 'SP.POP.0014.TO.ZS')
    value_name='Population Percentage' # Ini akan berisi nilainya (misal 15.2)
)

# Buat label yang lebih mudah dibaca
age_rename_dict = {
    'SP.POP.0014.TO.ZS': 'Ages 0-14',
    'SP.POP.1564.TO.ZS': 'Ages 15-64 (Productive)',
    'SP.POP.65UP.TO.ZS': 'Ages 65+ (Elderly)'
}
df_melted['Age Group'] = df_melted['Age Group Indicator'].map(age_rename_dict)

# Filter hanya tahun yang kita mau
years_to_plot = [2019, 2020, 2021]
df_filtered = df_melted[df_melted[col_year].isin(years_to_plot)].copy()

# Bersihkan data
df_filtered = df_filtered.dropna(subset=['Population Percentage', col_rate])

if df_filtered.empty:
    print("Error: Tidak ada data valid setelah di-melt dan dibersihkan.")
    exit()

print(f"Data 2019-2021 (3 kelompok usia) siap di-plot.")

# --- 4. Buat Visualisasi (Faceted Grid 3x3) ---
try:
    # Chart dasar
    base = alt.Chart(df_filtered).encode(
        x=alt.X('Population Percentage:Q', axis=alt.Axis(title='Population (%)')),
        y=alt.Y('Suicide Rate:Q', axis=alt.Axis(title='Suicide Rate'))
    )
    
    # Layer 1: Titik-titik scatter plot
    scatter_points = base.mark_point(opacity=0.6, filled=True).encode(
        tooltip=[
            alt.Tooltip(col_country, title='Country'),
            alt.Tooltip(col_year, title='Year'),
            alt.Tooltip('Age Group', title='Age Group'),
            alt.Tooltip('Population Percentage', title='Pop. (%)', format='.1f'),
            alt.Tooltip(col_rate, title='Suicide Rate', format='.1f')
        ]
    )
    
    # Layer 2: Garis regresi (tren)
    regression_line = base.transform_regression(
        'Population Percentage', col_rate, method='linear'
    ).mark_line(
        color='red',
        strokeWidth=2
    )

    # Gabungkan kedua layer
    chart = (scatter_points + regression_line).properties(
        width=220, # Buat tiap plot lebih kecil
        height=180
    ).interactive()

    # --- INI BAGIAN KUNCINYA (GRID 3x3) ---
    faceted_chart = chart.facet(
        # Buat 3 kolom, satu per TAHUN
        column=alt.Column('Year:O', header=alt.Header(titleOrient="bottom", labelOrient="bottom")),
        # Buat 3 baris, satu per KELOMPOK USIA
        row=alt.Column('Age Group:N', header=alt.Header(titleOrient="left", labelOrient="left"))
    ).resolve_scale(
        y='shared' # Sumbu Y sama untuk semua plot
    )

    # Simpan sebagai file JSON
    chart_file = 'slide3_age_vs_suicide_GRID_3x3.json'
    faceted_chart.save(chart_file)
    
    print(f"\n--- BERHASIL ---")
    print(f"Grafik Grid 3x3 'Age vs Suicide' (per tahun & usia) berhasil disimpan ke: {chart_file}")

except Exception as e:
    print(f"Error saat membuat chart: {e}")

--- Memulai script Plot 2: Age vs Suicide (Grid 3x3 Lengkap) ---
Menggunakan file: dataset_master_lengkap.csv
Berhasil memuat dataset_master_lengkap.csv
Semua kolom (3 Kelompok Usia, Suicide Rate, Year) ditemukan.
Data 2019-2021 (3 kelompok usia) siap di-plot.

--- BERHASIL ---
Grafik Grid 3x3 'Age vs Suicide' (per tahun & usia) berhasil disimpan ke: slide3_age_vs_suicide_GRID_3x3.json


c:\software\environments\deep_learning\environments\deep_learning_cv\Lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\software\environments\deep_learning\environments\deep_learning_cv\Lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\software\environments\deep_learning\environments\deep_learning_cv\Lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=Fals

In [3]:
import vl_convert as vlc
import json

# Nama file JSON input (yang baru saja kamu buat)
json_file_name = 'slide3_age_vs_suicide_GRID_3x3.json'

# Nama file PNG output
png_file_name = 'slide3_age_vs_suicide_GRID_3x3.png'

print(f"Mencoba memuat '{json_file_name}'...")

try:
    # 1. Muat file JSON
    with open(json_file_name, 'r') as f:
        chart_json = json.load(f)
    print("File JSON berhasil dimuat.")

    # 2. Konversi ke PNG
    print("Mengonversi ke PNG...")
    png_data = vlc.vegalite_to_png(vl_spec=chart_json, scale=2) # scale=2 untuk resolusi lebih tinggi
    print("Konversi PNG berhasil.")

    # 3. Simpan file PNG
    with open(png_file_name, "wb") as f:
        f.write(png_data)
    
    print(f"\n--- BERHASIL ---")
    print(f"Plot Grid 3x3 Usia berhasil disimpan sebagai: {png_file_name}")

except FileNotFoundError:
    print(f"ERROR: File tidak ditemukan: '{json_file_name}'")
    print("Pastikan kamu sudah menjalankan script sebelumnya untuk membuat file JSON.")
except Exception as e:
    print(f"\nTerjadi error: {e}")
    print("Pastikan kamu sudah menjalankan 'pip install vl-convert-python'")

Mencoba memuat 'slide3_age_vs_suicide_GRID_3x3.json'...
File JSON berhasil dimuat.
Mengonversi ke PNG...
Konversi PNG berhasil.

--- BERHASIL ---
Plot Grid 3x3 Usia berhasil disimpan sebagai: slide3_age_vs_suicide_GRID_3x3.png


In [4]:
import pandas as pd
import altair as alt

# --- 1. Tentukan Nama File ---
file_master = 'dataset_master_lengkap.csv'

print("--- Memulai script Plot 2: Age vs Suicide (Versi Interaktif) ---")
print(f"Menggunakan file: {file_master}")

# --- 2. Muat Data Gabungan ---
try:
    df_master = pd.read_csv(file_master)
    print(f"Berhasil memuat {file_master}")
except FileNotFoundError:
    print(f"Error: File '{file_master}' tidak ditemukan.")
    exit()
except Exception as e:
    print(f"Error saat memuat file: {e}")
    exit()

# --- 3. Proses Data (MELT Kolom Usia) ---
# (Langkah ini sama persis dengan kode 3x3 Grid sebelumnya)
col_rate = 'Suicide Rate'
col_year = 'Year'
col_country = 'Country Name'
age_cols = ['SP.POP.0014.TO.ZS', 'SP.POP.1564.TO.ZS', 'SP.POP.65UP.TO.ZS']

required_cols = age_cols + [col_rate, col_year, col_country]
if not all(col in df_master.columns for col in required_cols):
    print(f"Error: Tidak semua kolom ditemukan.")
    exit()

df_melted = df_master.melt(
    id_vars=[col_country, col_year, col_rate],
    value_vars=age_cols,
    var_name='Age Group Indicator',
    value_name='Population Percentage'
)

age_rename_dict = {
    'SP.POP.0014.TO.ZS': 'Ages 0-14',
    'SP.POP.1564.TO.ZS': 'Ages 15-64 (Productive)',
    'SP.POP.65UP.TO.ZS': 'Ages 65+ (Elderly)'
}
df_melted['Age Group'] = df_melted['Age Group Indicator'].map(age_rename_dict)

years_to_plot = [2019, 2020, 2021]
df_filtered = df_melted[df_melted[col_year].isin(years_to_plot)].copy()
df_filtered = df_filtered.dropna(subset=['Population Percentage', col_rate])
print("Data 2019-2021 (3 kelompok usia) siap di-plot.")

# --- 4. Buat Visualisasi (Interaktif dengan Slider & Dropdown) ---
try:
    # --- BUAT KONTROL INTERAKTIF ---
    
    # 1. Dropdown untuk Kelompok Usia
    age_groups = list(age_rename_dict.values())
    age_dropdown = alt.binding_select(options=age_groups, name='Age Group: ')
    select_age = alt.selection_point(fields=['Age Group'], bind=age_dropdown, value=age_groups[2]) # Default ke 65+

    # 2. Slider untuk Tahun
    year_slider = alt.binding_range(min=2019, max=2021, step=1, name='Year: ')
    select_year = alt.selection_point(fields=['Year'], bind=year_slider, value=2021) # Default ke 2021

    # --- BUAT CHART ---
    
    # Chart dasar (disaring oleh dropdown & slider)
    base = alt.Chart(df_filtered).add_params(
        select_age,
        select_year
    ).transform_filter(
        select_age
    ).transform_filter(
        select_year
    ).encode(
        x=alt.X('Population Percentage:Q', axis=alt.Axis(title='Population (%)')),
        y=alt.Y('Suicide Rate:Q', axis=alt.Axis(title='Suicide Rate (per 100k)'))
    )
    
    # Layer 1: Titik-titik scatter plot
    scatter_points = base.mark_point(opacity=0.7, filled=True).encode(
        tooltip=[
            alt.Tooltip(col_country, title='Country'),
            alt.Tooltip(col_year, title='Year'),
            alt.Tooltip('Age Group', title='Age Group'),
            alt.Tooltip('Population Percentage', title='Pop. (%)', format='.1f'),
            alt.Tooltip(col_rate, title='Suicide Rate', format='.1f')
        ]
    )
    
    # Layer 2: Garis regresi (tren)
    regression_line = base.transform_regression(
        'Population Percentage', col_rate, method='linear'
    ).mark_line(
        color='red',
        strokeWidth=2
    )

    # Gabungkan kedua layer dan tambahkan properti
    chart = (scatter_points + regression_line).properties(
        title='Suicide Rate vs. Population Demographics by Year',
        width=600,
        height=400
    )

    # Simpan sebagai file JSON
    chart_file = 'slide3_age_vs_suicide_INTERACTIVE.json'
    chart.save(chart_file)
    
    print(f"\n--- BERHASIL ---")
    print(f"Grafik Interaktif 'Age vs Suicide' (dengan slider & dropdown) berhasil disimpan ke: {chart_file}")

except Exception as e:
    print(f"Error saat membuat chart: {e}")

--- Memulai script Plot 2: Age vs Suicide (Versi Interaktif) ---
Menggunakan file: dataset_master_lengkap.csv
Berhasil memuat dataset_master_lengkap.csv
Data 2019-2021 (3 kelompok usia) siap di-plot.
Error saat membuat chart: module 'altair' has no attribute 'selection_point'


In [5]:
import pandas as pd
import altair as alt

# --- 1. Tentukan Nama File ---
file_master = 'dataset_master_lengkap.csv'

print("--- Memulai script Plot 2: Age vs Suicide (Perbaikan Versi Altair) ---")
print(f"Menggunakan file: {file_master}")

# --- 2. Muat Data Gabungan ---
try:
    df_master = pd.read_csv(file_master)
    print(f"Berhasil memuat {file_master}")
except FileNotFoundError:
    print(f"Error: File '{file_master}' tidak ditemukan.")
    exit()
except Exception as e:
    print(f"Error saat memuat file: {e}")
    exit()

# --- 3. Proses Data (MELT Kolom Usia) ---
# (Langkah ini sama persis dengan kode 3x3 Grid sebelumnya)
col_rate = 'Suicide Rate'
col_year = 'Year'
col_country = 'Country Name'
age_cols = ['SP.POP.0014.TO.ZS', 'SP.POP.1564.TO.ZS', 'SP.POP.65UP.TO.ZS']

required_cols = age_cols + [col_rate, col_year, col_country]
if not all(col in df_master.columns for col in required_cols):
    print(f"Error: Tidak semua kolom ditemukan.")
    exit()

df_melted = df_master.melt(
    id_vars=[col_country, col_year, col_rate],
    value_vars=age_cols,
    var_name='Age Group Indicator',
    value_name='Population Percentage'
)

age_rename_dict = {
    'SP.POP.0014.TO.ZS': 'Ages 0-14',
    'SP.POP.1564.TO.ZS': 'Ages 15-64 (Productive)',
    'SP.POP.65UP.TO.ZS': 'Ages 65+ (Elderly)'
}
df_melted['Age Group'] = df_melted['Age Group Indicator'].map(age_rename_dict)

years_to_plot = [2019, 2020, 2021]
df_filtered = df_melted[df_melted[col_year].isin(years_to_plot)].copy()
df_filtered = df_filtered.dropna(subset=['Population Percentage', col_rate])
print("Data 2019-2021 (3 kelompok usia) siap di-plot.")

# --- 4. Buat Visualisasi (Interaktif dengan Slider & Dropdown) ---
try:
    # --- BUAT KONTROL INTERAKTIF ---
    
    # 1. Dropdown untuk Kelompok Usia
    age_groups = list(age_rename_dict.values())
    age_dropdown = alt.binding_select(options=age_groups, name='Age Group: ')
    
    # --- PERBAIKAN DI SINI ---
    # Mengganti alt.selection_point -> alt.selection_single
    select_age = alt.selection_single(fields=['Age Group'], bind=age_dropdown, value=age_groups[2])

    # 2. Slider untuk Tahun
    year_slider = alt.binding_range(min=2019, max=2021, step=1, name='Year: ')
    
    # --- PERBAIKAN DI SINI ---
    # Mengganti alt.selection_point -> alt.selection_single
    select_year = alt.selection_single(fields=['Year'], bind=year_slider, value=2021)

    # --- BUAT CHART ---
    
    base = alt.Chart(df_filtered).add_params(
        select_age,
        select_year
    ).transform_filter(
        select_age
    ).transform_filter(
        select_year
    ).encode(
        x=alt.X('Population Percentage:Q', axis=alt.Axis(title='Population (%)')),
        y=alt.Y('Suicide Rate:Q', axis=alt.Axis(title='Suicide Rate (per 100k)'))
    )
    
    scatter_points = base.mark_point(opacity=0.7, filled=True).encode(
        tooltip=[
            alt.Tooltip(col_country, title='Country'),
            alt.Tooltip(col_year, title='Year'),
            alt.Tooltip('Age Group', title='Age Group'),
            alt.Tooltip('Population Percentage', title='Pop. (%)', format='.1f'),
            alt.Tooltip(col_rate, title='Suicide Rate', format='.1f')
        ]
    )
    
    regression_line = base.transform_regression(
        'Population Percentage', col_rate, method='linear'
    ).mark_line(
        color='red',
        strokeWidth=2
    )

    chart = (scatter_points + regression_line).properties(
        title='Suicide Rate vs. Population Demographics by Year',
        width=600,
        height=400
    )

    # Simpan sebagai file JSON
    chart_file = 'slide3_age_vs_suicide_INTERACTIVE.json'
    chart.save(chart_file)
    
    print(f"\n--- BERHASIL ---")
    print(f"Grafik Interaktif 'Age vs Suicide' (dengan slider & dropdown) berhasil disimpan ke: {chart_file}")

except Exception as e:
    print(f"Error saat membuat chart: {e}")

--- Memulai script Plot 2: Age vs Suicide (Perbaikan Versi Altair) ---
Menggunakan file: dataset_master_lengkap.csv
Berhasil memuat dataset_master_lengkap.csv
Data 2019-2021 (3 kelompok usia) siap di-plot.
Error saat membuat chart: Invalid specification

        altair.vegalite.v4.schema.core.SelectionDef->1->bind, validating 'anyOf'

        {'input': 'select', 'options': ['Ages 0-14', 'Ages 15-64 (Productive)', 'Ages 65+ (Elderly)'], 'name': 'Age Group: '} is not valid under any of the given schemas
        


In [8]:
import pandas as pd
import altair as alt

# --- 1. Tentukan Nama File ---
file_master = 'dataset_master_lengkap.csv'

print("--- Memulai script Plot 2: Age vs Suicide (Perbaikan v4 Final: add_selection) ---")
print(f"Menggunakan file: {file_master}")

# --- 2. Muat Data Gabungan ---
try:
    df_master = pd.read_csv(file_master)
    print(f"Berhasil memuat {file_master}")
except FileNotFoundError:
    print(f"Error: File '{file_master}' tidak ditemukan.")
    exit()
except Exception as e:
    print(f"Error saat memuat file: {e}")
    exit()

# --- 3. Proses Data (MELT Kolom Usia) ---
col_rate = 'Suicide Rate'
col_year = 'Year'
col_country = 'Country Name'
age_cols = ['SP.POP.0014.TO.ZS', 'SP.POP.1564.TO.ZS', 'SP.POP.65UP.TO.ZS']

required_cols = age_cols + [col_rate, col_year, col_country]
if not all(col in df_master.columns for col in required_cols):
    print(f"Error: Tidak semua kolom ditemukan.")
    exit()

df_melted = df_master.melt(
    id_vars=[col_country, col_year, col_rate],
    value_vars=age_cols,
    var_name='Age Group Indicator',
    value_name='Population Percentage'
)

age_rename_dict = {
    'SP.POP.0014.TO.ZS': 'Ages 0-14',
    'SP.POP.1564.TO.ZS': 'Ages 15-64 (Productive)',
    'SP.POP.65UP.TO.ZS': 'Ages 65+ (Elderly)'
}
df_melted['Age Group'] = df_melted['Age Group Indicator'].map(age_rename_dict)

years_to_plot = [2019, 2020, 2021]
df_filtered = df_melted[df_melted[col_year].isin(years_to_plot)].copy()
df_filtered = df_filtered.dropna(subset=['Population Percentage', col_rate])
print("Data 2019-2021 (3 kelompok usia) siap di-plot.")

# --- 4. Buat Visualisasi (Interaktif dengan Slider & Dropdown) ---
try:
    # --- BUAT KONTROL INTERAKTIF ---
    
    # 1. Dropdown untuk Kelompok Usia
    age_groups = list(age_rename_dict.values())
    age_dropdown = alt.binding_select(options=age_groups, name='Age Group: ')
    
    select_age = alt.selection_single(
        fields=['Age Group'], 
        bind=age_dropdown, 
        init={'Age Group': age_groups[2]} # Default ke 65+
    )

    # 2. Slider untuk Tahun
    year_slider = alt.binding_range(min=2019, max=2021, step=1, name='Year: ')
    
    select_year = alt.selection_single(
        fields=['Year'], 
        bind=year_slider, 
        init={'Year': 2021} # Default ke 2021
    )

    # --- BUAT CHART ---
    
    # --- PERBAIKAN DI SINI (add_params -> add_selection) ---
    base = alt.Chart(df_filtered).add_selection(
        select_age,
        select_year
    ).transform_filter(
        select_age
    ).transform_filter(
        select_year
    ).encode(
        x=alt.X('Population Percentage:Q', axis=alt.Axis(title='Population (%)')),
        y=alt.Y('Suicide Rate:Q', axis=alt.Axis(title='Suicide Rate (per 100k)'))
    )
    
    scatter_points = base.mark_point(opacity=0.7, filled=True).encode(
        tooltip=[
            alt.Tooltip(col_country, title='Country'),
            alt.Tooltip(col_year, title='Year'),
            alt.Tooltip('Age Group', title='Age Group'),
            alt.Tooltip('Population Percentage', title='Pop. (%)', format='.1f'),
            alt.Tooltip(col_rate, title='Suicide Rate', format='.1f')
        ]
    )
    
    regression_line = base.transform_regression(
        'Population Percentage', col_rate, method='linear'
    ).mark_line(
        color='red',
        strokeWidth=2
    )

    chart = (scatter_points + regression_line).properties(
        title='Suicide Rate vs. Population Demographics by Year',
        width=600,
        height=400
    )

    # Simpan sebagai file JSON
    chart_file = 'slide3_age_vs_suicide_INTERACTIVE.json'
    chart.save(chart_file)
    
    print(f"\n--- BERHASIL ---")
    print(f"Grafik Interaktif 'Age vs Suicide' (dengan slider & dropdown) berhasil disimpan ke: {chart_file}")

except Exception as e:
    print(f"Error saat membuat chart: {e}")

--- Memulai script Plot 2: Age vs Suicide (Perbaikan v4 Final: add_selection) ---
Menggunakan file: dataset_master_lengkap.csv
Berhasil memuat dataset_master_lengkap.csv
Data 2019-2021 (3 kelompok usia) siap di-plot.

--- BERHASIL ---
Grafik Interaktif 'Age vs Suicide' (dengan slider & dropdown) berhasil disimpan ke: slide3_age_vs_suicide_INTERACTIVE.json


c:\software\environments\deep_learning\environments\deep_learning_cv\Lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\software\environments\deep_learning\environments\deep_learning_cv\Lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\software\environments\deep_learning\environments\deep_learning_cv\Lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=Fals

In [9]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Mengatur 'template' default agar background-nya putih
pio.templates.default = "plotly_white"

print("--- Memulai script Plot 2: Age vs Suicide (Versi PLOTLY) ---")

# --- 1. Tentukan Nama File ---
file_master = 'dataset_master_lengkap.csv'

# --- 2. Muat Data Gabungan ---
try:
    df_master = pd.read_csv(file_master)
    print(f"Berhasil memuat {file_master}")
except FileNotFoundError:
    print(f"Error: File '{file_master}' tidak ditemukan.")
    exit()
except Exception as e:
    print(f"Error saat memuat file: {e}")
    exit()

# --- 3. Proses Data (MELT Kolom Usia) ---
# (Langkah ini sama persis dengan kode 3x3 Grid sebelumnya)
col_rate = 'Suicide Rate'
col_year = 'Year'
col_country = 'Country Name'
age_cols = ['SP.POP.0014.TO.ZS', 'SP.POP.1564.TO.ZS', 'SP.POP.65UP.TO.ZS']

required_cols = age_cols + [col_rate, col_year, col_country]
if not all(col in df_master.columns for col in required_cols):
    print(f"Error: Tidak semua kolom ditemukan.")
    exit()

df_melted = df_master.melt(
    id_vars=[col_country, col_year, col_rate],
    value_vars=age_cols,
    var_name='Age Group Indicator',
    value_name='Population Percentage'
)

age_rename_dict = {
    'SP.POP.0014.TO.ZS': 'Ages 0-14',
    'SP.POP.1564.TO.ZS': 'Ages 15-64 (Productive)',
    'SP.POP.65UP.TO.ZS': 'Ages 65+ (Elderly)'
}
df_melted['Age Group'] = df_melted['Age Group Indicator'].map(age_rename_dict)

years_to_plot = [2019, 2020, 2021]
df_filtered = df_melted[df_melted[col_year].isin(years_to_plot)].copy()
df_filtered = df_filtered.dropna(subset=['Population Percentage', col_rate])
print("Data 2019-2021 (3 kelompok usia) siap di-plot.")

# --- 4. Buat Visualisasi (Plotly Facet + Slider) ---
try:
    print("Membuat peta Plotly...")
    fig = px.scatter(
        df_filtered,
        x="Population Percentage",
        y="Suicide Rate",
        animation_frame="Year",         # Ini membuat SLIDER TAHUN
        facet_row="Age Group",          # Ini membuat 3 plot vertikal
        trendline="ols",                # Ini menambahkan garis regresi (linear)
        hover_name="Country Name",
        title="Suicide Rate vs. Population Demographics by Year"
    )
    
    # Atur kategori urutan baris
    fig.update_layout(
        height=900, # Buat lebih tinggi agar 3 plot muat
        xaxis_title="Population (%)",
        yaxis_title="Suicide Rate (per 100k)"
    )
    # Ganti nama label facet
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    
    # --- 5. Menyimpan Peta ---
    
    # Cara 1: Simpan sebagai HTML (Interaktif, Pasti Berhasil)
    html_file = 'slide3_age_vs_suicide_INTERACTIVE.html'
    fig.write_html(html_file)
    print(f"\n--- BERHASIL (HTML) ---")
    print(f"Plot INTERAKTIF disimpan ke: {html_file}")
    print("Buka file ini di browser untuk melihat slider dan 3 plotnya.")

    # Cara 2: Coba Simpan sebagai PNG (Statis)
    try:
        png_file = 'slide3_age_vs_suicide.png'
        # Kamu harus install kaleido dulu: pip install kaleido
        fig.write_image(png_file, width=800, height=1000)
        print(f"\n--- BERHASIL (PNG) ---")
        print(f"Gambar plot (PNG) berhasil disimpan ke: {png_file}")
    except ValueError as e_png:
        print(f"\n--- INFO (PNG Gagal) ---")
        print(f"Gagal menyimpan PNG: {e_png}")
        print("Ini biasanya karena 'kaleido' belum ter-install.")
        print("Jalankan 'pip install kaleido' di terminalmu lalu coba lagi.")
        
except Exception as e_fig:
    print(f"Error membuat Peta Plotly: {e_fig}")

--- Memulai script Plot 2: Age vs Suicide (Versi PLOTLY) ---
Berhasil memuat dataset_master_lengkap.csv
Data 2019-2021 (3 kelompok usia) siap di-plot.
Membuat peta Plotly...

--- BERHASIL (HTML) ---
Plot INTERAKTIF disimpan ke: slide3_age_vs_suicide_INTERACTIVE.html
Buka file ini di browser untuk melihat slider dan 3 plotnya.

--- INFO (PNG Gagal) ---
Gagal menyimpan PNG: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Ini biasanya karena 'kaleido' belum ter-install.
Jalankan 'pip install kaleido' di terminalmu lalu coba lagi.


# 3. plot ke tiga yg undep

In [10]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Mengatur 'template' default agar background-nya putih
pio.templates.default = "plotly_white"

print("--- Memulai script Plot 3: Unemployment vs Suicide (Versi PLOTLY) ---")

# --- 1. Tentukan Nama File ---
file_master = 'dataset_master_lengkap.csv'

# --- 2. Muat Data Gabungan ---
try:
    df_master = pd.read_csv(file_master)
    print(f"Berhasil memuat {file_master}")
except FileNotFoundError:
    print(f"Error: File '{file_master}' tidak ditemukan.")
    exit()
except Exception as e:
    print(f"Error saat memuat file: {e}")
    exit()

# --- 3. Proses Data ---
col_unemployment = 'SL.UEM.TOTL.ZS' # Kolom Unemployment
col_rate = 'Suicide Rate'
col_year = 'Year'
col_country = 'Country Name'

# Cek apakah semua kolom ada
required_cols = [col_unemployment, col_rate, col_year, col_country]
if not all(col in df_master.columns for col in required_cols):
    print(f"Error: Tidak semua kolom ditemukan di {file_master}.")
    print(f"Kolom yang ada: {df_master.columns.tolist()}")
    exit()
else:
    print("Semua kolom (Unemployment, Suicide Rate, Year) ditemukan.")

# Filter hanya tahun yang kita mau
years_to_plot = [2019, 2020, 2021]
df_filtered = df_master[df_master[col_year].isin(years_to_plot)].copy()

# Bersihkan data, hapus baris yang datanya kosong untuk plot ini
df_filtered = df_filtered.dropna(subset=[col_unemployment, col_rate])

if df_filtered.empty:
    print("Error: Tidak ada data valid untuk tahun 2019-2021 setelah dibersihkan.")
    exit()

print(f"Data 2019-2021 siap untuk di-plot (total {len(df_filtered)} data poin).")


# --- 4. Buat Visualisasi (Plotly Scatter + Slider) ---
try:
    print("Membuat plot Plotly...")
    fig = px.scatter(
        df_filtered,
        x=col_unemployment,
        y=col_rate,
        animation_frame="Year",         # Ini membuat SLIDER TAHUN
        trendline="ols",                # Ini menambahkan garis regresi (linear)
        hover_name="Country Name",
        title="Suicide Rate vs. Unemployment Rate by Year"
    )
    
    # Atur label sumbu
    fig.update_layout(
        xaxis_title="Unemployment Rate (%)",
        yaxis_title="Suicide Rate (per 100k)"
    )
    
    # --- 5. Menyimpan Plot ---
    
    # Cara 1: Simpan sebagai HTML (Interaktif, Pasti Berhasil)
    html_file = 'slide3_unemployment_vs_suicide_INTERACTIVE.html'
    fig.write_html(html_file)
    print(f"\n--- BERHASIL (HTML) ---")
    print(f"Plot INTERAKTIF disimpan ke: {html_file}")
    print("Buka file ini di browser untuk melihat slider-nya.")

    # Cara 2: Coba Simpan sebagai PNG (Statis)
    try:
        png_file = 'slide3_unemployment_vs_suicide.png'
        # Kamu harus install kaleido dulu: pip install kaleido
        fig.write_image(png_file, width=900, height=500)
        print(f"\n--- BERHASIL (PNG) ---")
        print(f"Gambar plot (PNG) berhasil disimpan ke: {png_file}")
    except ValueError as e_png:
        print(f"\n--- INFO (PNG Gagal) ---")
        print(f"Gagal menyimpan PNG: {e_png}")
        print("Ini biasanya karena 'kaleido' belum ter-install.")
        print("Jalankan 'pip install kaleido' di terminalmu lalu coba lagi.")
        
except Exception as e_fig:
    print(f"Error membuat Peta Plotly: {e_fig}")

--- Memulai script Plot 3: Unemployment vs Suicide (Versi PLOTLY) ---
Berhasil memuat dataset_master_lengkap.csv
Semua kolom (Unemployment, Suicide Rate, Year) ditemukan.
Data 2019-2021 siap untuk di-plot (total 465 data poin).
Membuat plot Plotly...

--- BERHASIL (HTML) ---
Plot INTERAKTIF disimpan ke: slide3_unemployment_vs_suicide_INTERACTIVE.html
Buka file ini di browser untuk melihat slider-nya.

--- INFO (PNG Gagal) ---
Gagal menyimpan PNG: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Ini biasanya karena 'kaleido' belum ter-install.
Jalankan 'pip install kaleido' di terminalmu lalu coba lagi.


In [11]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Mengatur 'template' default agar background-nya putih
pio.templates.default = "plotly_white"

print("--- Memulai script Plot 3: Unemployment (Versi Warna Benua) ---")

# --- 1. Tentukan Nama File ---
file_master = 'dataset_master_lengkap.csv'
file_metadata = 'WDICountry.csv' # File metadata untuk data 'Region'

# --- 2. Muat Data ---
try:
    df_master = pd.read_csv(file_master)
    df_meta = pd.read_csv(file_metadata)
    print(f"Berhasil memuat {file_master} dan {file_metadata}")
except FileNotFoundError as e:
    print(f"Error: File tidak ditemukan. {e}")
    exit()
except Exception as e:
    print(f"Error saat memuat file: {e}")
    exit()

# --- 3. Gabungkan Data Master dengan Metadata (untuk dapat 'Region') ---
# Siapkan metadata: ambil kolom 'Table Name' (nama bersih) dan 'Region'
df_meta_processed = df_meta[['Table Name', 'Region']].dropna()

# Gabungkan!
# 'Country Name' dari file master-mu
# 'Table Name' dari file metadata
df_with_region = pd.merge(
    df_master,
    df_meta_processed,
    left_on='Country Name',
    right_on='Table Name',
    how='left' # 'left' = tetap simpan semua data suicide, walau ada yg ga cocok region-nya
)
print("Berhasil menggabungkan data master dengan metadata Region.")

# --- 4. Proses Data ---
col_unemployment = 'SL.UEM.TOTL.ZS' # Kolom Unemployment
col_rate = 'Suicide Rate'
col_year = 'Year'
col_country = 'Country Name'
col_region = 'Region' # Kolom baru kita!

# Cek apakah semua kolom ada
required_cols = [col_unemployment, col_rate, col_year, col_country, col_region]
if not all(col in df_with_region.columns for col in required_cols):
    print(f"Error: Gagal menggabungkan atau kolom tidak ditemukan.")
    exit()
else:
    print("Semua kolom (termasuk Region) ditemukan.")

# Filter hanya tahun yang kita mau
years_to_plot = [2019, 2020, 2021]
df_filtered = df_with_region[df_with_region[col_year].isin(years_to_plot)].copy()

# Bersihkan data, hapus baris yang datanya kosong untuk plot ini
df_filtered = df_filtered.dropna(subset=[col_unemployment, col_rate, col_region])

if df_filtered.empty:
    print("Error: Tidak ada data valid untuk tahun 2019-2021 setelah dibersihkan.")
    exit()

print(f"Data 2019-2021 (dengan Region) siap untuk di-plot.")


# --- 5. Buat Visualisasi (Plotly Scatter + Slider + WARNA) ---
try:
    print("Membuat plot Plotly...")
    fig = px.scatter(
        df_filtered,
        x=col_unemployment,
        y=col_rate,
        animation_frame="Year",         # Slider Tahun
        trendline="ols",                # Garis regresi
        color=col_region,               # <-- INI YANG BARU: Warna berdasarkan Benua
        hover_name="Country Name",
        title="Suicide Rate vs. Unemployment Rate by Year and Continent"
    )
    
    # Atur label sumbu
    fig.update_layout(
        xaxis_title="Unemployment Rate (%)",
        yaxis_title="Suicide Rate (per 100k)",
        legend_title="Continent" # Judul legenda
    )
    
    # --- 6. Menyimpan Plot ---
    
    # Simpan sebagai HTML (Interaktif, Pasti Berhasil)
    html_file = 'slide3_unemployment_vs_suicide_WARNA.html'
    fig.write_html(html_file)
    print(f"\n--- BERHASIL (HTML) ---")
    print(f"Plot INTERAKTIF (berwarna) disimpan ke: {html_file}")

    # Coba Simpan sebagai PNG (Statis)
    try:
        png_file = 'slide3_unemployment_vs_suicide_WARNA.png'
        fig.write_image(png_file, width=1000, height=550) # Dibuat lebih lebar untuk legenda
        print(f"\n--- BERHASIL (PNG) ---")
        print(f"Gambar plot (PNG berwarna) berhasil disimpan ke: {png_file}")
    except ValueError as e_png:
        print(f"\n--- INFO (PNG Gagal) ---")
        print(f"Gagal menyimpan PNG: {e_png}")
        print("Pastikan 'kaleido' sudah ter-install: pip install kaleido")
        
except Exception as e_fig:
    print(f"Error membuat Peta Plotly: {e_fig}")

--- Memulai script Plot 3: Unemployment (Versi Warna Benua) ---
Berhasil memuat dataset_master_lengkap.csv dan WDICountry.csv
Berhasil menggabungkan data master dengan metadata Region.
Semua kolom (termasuk Region) ditemukan.
Data 2019-2021 (dengan Region) siap untuk di-plot.
Membuat plot Plotly...

--- BERHASIL (HTML) ---
Plot INTERAKTIF (berwarna) disimpan ke: slide3_unemployment_vs_suicide_WARNA.html

--- INFO (PNG Gagal) ---
Gagal menyimpan PNG: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Pastikan 'kaleido' sudah ter-install: pip install kaleido


# 1. Gender vs suicide rate

In [12]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Mengatur 'template' default agar background-nya putih
pio.templates.default = "plotly_white"

print("--- Memulai script Plot 4: Gender vs Suicide (Versi PLOTLY) ---")

# --- 1. Tentukan Nama File ---
# Kita pakai file suicide ASLI, bukan file master
file_suicide = 'suicide-rate-by-country-2025.csv'

# --- 2. Muat Data ---
try:
    df_suicide_raw = pd.read_csv(file_suicide)
    print(f"Berhasil memuat {file_suicide}")
except FileNotFoundError:
    print(f"Error: File '{file_suicide}' tidak ditemukan.")
    exit()
except Exception as e:
    print(f"Error saat memuat file: {e}")
    exit()

# --- 3. Proses Data (Melt) ---
# Tentukan kolom-kolom yang akan kita ambil
cols_to_melt = [
    'SuicideRateMaleCountries_2019', 'SuicideRateMaleCountries_2020', 'SuicideRateMaleCountries_2021',
    'SuicideRateFemaleCountries_2019', 'SuicideRateFemaleCountries_2020', 'SuicideRateFemaleCountries_2021'
]

# Cek apakah kolomnya ada
existing_cols = [col for col in cols_to_melt if col in df_suicide_raw.columns]
if len(existing_cols) < 6:
    print(f"Error: Tidak semua kolom Male/Female ditemukan di file.")
    exit()

# Proses MELT
df_long = df_suicide_raw.melt(
    id_vars=['country'],
    value_vars=existing_cols,
    var_name='Indicator',
    value_name='Suicide Rate'
)

# --- 4. Ekstrak Info Tahun dan Gender ---
# Ekstrak Tahun
df_long['Year'] = df_long['Indicator'].str.extract(r'(\d{4})').astype(int)

# Ekstrak Gender
# Buat fungsi untuk menentukan gender
def get_gender(indicator_name):
    if 'Male' in indicator_name:
        return 'Male'
    elif 'Female' in indicator_name:
        return 'Female'
    return 'Unknown'

df_long['Gender'] = df_long['Indicator'].apply(get_gender)

# Bersihkan data
df_long['Suicide Rate'] = pd.to_numeric(df_long['Suicide Rate'], errors='coerce')
df_final = df_long.dropna(subset=['Suicide Rate'])
print("Data suicide (Male/Female) berhasil di-MELT.")

# --- 5. Hitung Rata-Rata Global per Gender per Tahun ---
df_agg = df_final.groupby(['Year', 'Gender'])['Suicide Rate'].mean().reset_index()

print("Data rata-rata global per gender:")
print(df_agg.head(6))


# --- 6. Buat Visualisasi (Plotly Line Chart) ---
try:
    print("Membuat plot Plotly...")
    fig = px.line(
        df_agg,
        x="Year",
        y="Suicide Rate",
        color="Gender",           # Ini akan membuat 2 garis (Male & Female)
        markers=True,             # Tambahkan titik di tiap data poin
        title="Global Suicide Rate by Gender (2019-2021)"
    )
    
    # Atur label sumbu
    fig.update_layout(
        xaxis_title="Year",
        yaxis_title="Average Suicide Rate (per 100k)",
        legend_title="Gender"
    )
    # Pastikan sumbu X hanya menampilkan 2019, 2020, 2021
    fig.update_xaxes(tickmode='linear')
    
    # --- 7. Menyimpan Plot ---
    
    # Cara 1: Simpan sebagai HTML (Interaktif, Pasti Berhasil)
    html_file = 'slide3_gender_vs_suicide_INTERACTIVE.html'
    fig.write_html(html_file)
    print(f"\n--- BERHASIL (HTML) ---")
    print(f"Plot INTERAKTIF disimpan ke: {html_file}")

    # Cara 2: Coba Simpan sebagai PNG (Statis)
    try:
        png_file = 'slide3_gender_vs_suicide.png'
        fig.write_image(png_file, width=800, height=500)
        print(f"\n--- BERHASIL (PNG) ---")
        print(f"Gambar plot (PNG) berhasil disimpan ke: {png_file}")
    except ValueError as e_png:
        print(f"\n--- INFO (PNG Gagal) ---")
        print(f"Gagal menyimpan PNG: {e_png}")
        print("Pastikan 'kaleido' sudah ter-install: pip install kaleido")
        
except Exception as e_fig:
    print(f"Error membuat Peta Plotly: {e_fig}")

--- Memulai script Plot 4: Gender vs Suicide (Versi PLOTLY) ---
Berhasil memuat suicide-rate-by-country-2025.csv
Data suicide (Male/Female) berhasil di-MELT.
Data rata-rata global per gender:
   Year  Gender  Suicide Rate
0  2019  Female      4.348352
1  2019    Male     14.704420
2  2020  Female      4.398670
3  2020    Male     14.894187
4  2021  Female      4.356010
5  2021    Male     14.774039
Membuat plot Plotly...

--- BERHASIL (HTML) ---
Plot INTERAKTIF disimpan ke: slide3_gender_vs_suicide_INTERACTIVE.html

--- INFO (PNG Gagal) ---
Gagal menyimpan PNG: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido

Pastikan 'kaleido' sudah ter-install: pip install kaleido
